# BEAST Analysis Notebook

---

# 0. SETUP

In [2]:
import os
import pandas as pd
import seaborn as sns
from Bio import Phylo, AlignIO
from functions import *

## Paths

In [3]:
log_dir  = "/mnt/c/Users/ktmea/Projects/plague-phylogeography-projects/main/beast/all/chromosome/clade"
tree_dir = "/mnt/c/Users/ktmea/Projects/plague-phylogeography-projects/main/beast/all/chromosome/clade/summary_trees_noHyperPrior/relaxed_clock"
metadata_path = "/mnt/c/Users/ktmea/Projects/plague-phylogeography-projects/main/iqtree/all/chromosome/full/filter5/filter-taxa/metadata.tsv"
beast_dir = "/mnt/c/Users/ktmea/Projects/plague-phylogeography-projects/main/beast/all/chromosome/clade"

auspice_config_path = "/mnt/c/Users/ktmea/Projects/plague-phylogeography-projects/main/config/auspice_config.json"
out_path_colors = "/mnt/c/Users/ktmea/Projects/plague-phylogeography-projects/main/augur/all/chromosome/full/filter5/colors.tsv"
out_path_latlon = "/mnt/c/Users/ktmea/Projects/plague-phylogeography-projects/main/augur/all/chromosome/full/filter5/latlon.tsv"

# ------------------------------------------
# Alignment
constant_sites_path   = "/mnt/c/Users/ktmea/Projects/plague-phylogeography-projects/main/snippy_multi/all/chromosome/full/snippy-multi.constant_sites.txt"
aln_path              = "/mnt/c/Users/ktmea/Projects/plague-phylogeography-projects/main/iqtree/all/chromosome/full/filter5/filter-sites/snippy-multi.snps.aln"

## Variables

In [4]:
# ------------------------------------------
BRANCH_LIST = {
    "1.ORI" : ["1.ORI1", "1.ORI2", "1.ORI3"],
    "1.IN": ["1.IN1","1.IN2","1.IN3"],  
    "1.ANT": ["1.ANT1"], 
    "1.PRE" : ["1.PRE0","1.PRE1", "1.PRE2", "1.PRE3"],
    "2.MED": ["2.MED0", "2.MED1","2.MED2","2.MED3" ],      
    "2.ANT": ["2.ANT1","2.ANT2","2.ANT3" ],    
    "4.ANT": ["4.ANT1" ],       
    "3.ANT": ["3.ANT1", "3.ANT2" ],  
    "0.ANT": ["0.ANT1", "0.ANT2","0.ANT3","0.ANT5"],         
    "0.ANT4" : ["0.ANT4"], 
    "0.PE": ["0.PE2", "0.PE4m", "0.PE4m", "0.PE4t", "0.PE4a", "0.PE5", "0.PE7", "0.PE8", "0.PE10"],   
    "0.PRE": ["0.PRE1", "0.PRE2"],        
}

NUM_STATES = 10

NO_DATA_CHAR = "NA"
JSON_INDENT=2

# ------------------------------------------
# Alignment
with open(constant_sites_path) as infile:
    data = infile.read().strip().split(",")
    constant_sites = sum([int(count) for count in data])

aln = AlignIO.read(aln_path, "fasta")
variant_sites = len(aln[0].seq)
SEQ_LEN = constant_sites + variant_sites

---

# 1. IMPORT

## Metadata

In [5]:
metadata_df = pd.read_csv(metadata_path, sep='\t')
metadata_df.set_index(metadata_df.columns[0], inplace=True)
metadata_df.fillna(NO_DATA_CHAR, inplace=True)

display(metadata_df)

,strain,date,date_bp,country,province,country_lat,country_lon,province_lat,province_lon,biovar,...,biosample_accession,biosample_comment,branch_number,continent,date_mean,date_bp_mean,date_err,lat,lon,host_human
sample,,,,,,,,,,,,,,,,,,,,,
Reference,CO92,1992,-29,United States of America,Colorado,39.783730,-100.445882,38.7252,-105.608,Orientalis,...,SAMEA1705942,KEEP: Assembly Modern Reference,1,North America,1992.0,29.0,0.0,38.725178,-105.607716,Human
GCA_009909635.1_ASM990963v1_genomic,9_10,1923.0,-98,Russia,Rostov Oblast,64.686314,97.745306,47.6222,40.7958,Medievalis,...,SAMN13632815,KEEP: Assembly Modern,2,Europe,1923.0,98.0,0.0,47.622245,40.795794,Human
GCA_009669545.1_ASM966954v1_genomic,42126,2006.0,-15,China,Xinjiang,35.000074,104.999927,42.4805,85.4633,Antiqua,...,SAMN07722925,KEEP: Assembly Modern,0,Asia,2006.0,15.0,0.0,42.480495,85.463346,Non-Human
GCA_009669555.1_ASM966955v1_genomic,42123,2005.0,-16,China,Xinjiang,35.000074,104.999927,42.4805,85.4633,Antiqua,...,SAMN07722924,KEEP: Assembly Modern,0,Asia,2005.0,16.0,0.0,42.480495,85.463346,Non-Human
GCA_009669565.1_ASM966956v1_genomic,42118,2005.0,-16,China,Xinjiang,35.000074,104.999927,42.4805,85.4633,Antiqua,...,SAMN07722923,KEEP: Assembly Modern,0,Asia,2005.0,16.0,0.0,42.480495,85.463346,Non-Human
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SAMEA7313243_45,Azov38,[1400:1700],[-621:-321],Russia,Rostov Oblast,64.686314,97.745306,47.6222,40.7958,Second Pandemic,...,SAMEA7313243_45,KEEP: SRA Ancient Combined Record,1,Europe,1550.0,471.0,150.0,47.622245,40.795794,Human
SAMEA7313246_49,Gdansk8,[1400:1700],[-621:-321],Poland,Pomeranian Voivodeship,52.215933,19.134422,54.2456,18.1099,Second Pandemic,...,SAMEA7313246_49,KEEP: SRA Ancient Combined Record,1,Europe,1550.0,471.0,150.0,54.245560,18.109900,Human
SAMEA6651390,AGU010,[1435:1477],[-586:-544],Lithuania,Vilnius County,55.350000,23.750000,54.8227,25.2495,Second Pandemic,...,SAMEA6651390,KEEP: SRA Ancient,1,Europe,1456.0,565.0,21.0,54.822692,25.249534,Human


## Most Recent Sampling Date

In [6]:
out_path = os.path.join(beast_dir, "most_recent_sampling_dates.tsv")
mrsd_dict = {}

with open(out_path, "w") as outfile:
    for branch in BRANCH_LIST:
        branch_df = metadata_df[metadata_df["branch_minor"].isin(BRANCH_LIST[branch])]
        max_date = max(branch_df["date_mean"])
        outfile.write("{}\t{}\n".format(branch, max_date))
        mrsd_dict[branch] = max_date

## Colors

In [22]:
colors_dict = {}

colors_df = pd.read_csv(out_path_colors, sep='\t', header=None)
colors_df.columns = ["state", "value", "color"]

for state in set(colors_df["state"]):
    state_df = colors_df[colors_df["state"] == state]    
    colors_dict[state] = {}
    
    for value,color in zip(state_df["value"], state_df["color"]):
        colors_dict[state][value] = color

print(colors_dict)

{'branch_major': {'0.PRE': '#8000ff', '0.PE': '#4c4ffc', '0.ANT': '#1996f3', '3.ANT': '#1acee3', '4.ANT': '#4df3ce', '2.ANT': '#80ffb4', '2.MED': '#b3f396', '1.PRE': '#e6ce74', '1.ANT': '#ff964f', '1.IN': '#ff4f28', '1.ORI': '#ff0000', '?': '#969696'}, 'province': {'Krasnoyarsk Krai': '#8000ff', 'Panevezys County': '#7b07ff', 'Pärnu maakond': '#760eff', 'Irkutsk Oblast': '#7215ff', 'Bavaria': '#6d1dff', 'Altai Krai': '#6924fe', 'Qinghai': '#642bfe', 'Republic of Dagestan': '#6032fe', 'Goranboy District': '#5b39fd', 'Syunik Province': '#5740fd', 'Shirak Province': '#5247fc', 'Gegharkunik Province': '#4d4dfc', 'Samtskhe-Javakheti': '#4954fb', 'Shahbuz Rayon': '#445bfb', 'Samara Oblast': '#4062fa', 'Sečuán': '#3b68f9', 'Bayankhongor': '#376ff9', 'Inner Mongolia': '#3275f8', 'Sughd Province': '#2e7bf7', 'Talas Region': '#2982f6', 'Altai Republic': '#2488f5', 'Govi-Altai': '#208ef4', 'Övörkhangai': '#1b94f3', 'Ömnögovi': '#1799f2', 'Bayan-Ölgii': '#129ff1', 'Khovd': '#0ea5ef', 'Xinjiang': '

DEPRECATED: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



## Tree Files

In [8]:
# Construct a dictionary to hold the trees
tree_dict = {}

for branch in BRANCH_LIST:
    print("Branch:", branch)
    tree_dict[branch] = {} 
    for filename in os.listdir(tree_dir):
        if not filename.endswith(".tre"): continue
        filepath = os.path.join(tree_dir, filename)
        if branch in filename:            
            # Add tree files to dict
            tree_dict[branch]["tree_file_raw"] = filepath 
            tree_dict[branch]["tree_file_edit"] = os.path.join(tree_dir, branch + ".nex")
            tree_dict[branch]["sample_rename"] = {}
            
            # Read in raw tree to deal with dashes
            with open(tree_dict[branch]["tree_file_raw"],  "r") as infile:                    
                with open(tree_dict[branch]["tree_file_edit"], "w") as outfile:           
                    raw_tree = infile.read()
                    # Remove quotations if they exist
                    raw_tree = raw_tree.replace("'","")
                    
                    # Split into lines to iterate over
                    raw_tree_lines = raw_tree.split("\n")
                    
                    # By default don't parse a line for dashes
                    taxa_line = False
                    
                    for line in raw_tree_lines: 
                                
                        if "TREE" not in line and "-" in line:
                            
                            if len(line.split(" ")) == 1:
                                name_dashes = line.strip()
                                name_no_dashes = name_dashes.replace("-","_")
                                tree_dict[branch]["sample_rename"][name_no_dashes] = name_dashes
                            line = line.replace("-","_")
                                
                        outfile.write(line + "\n")
            
            # Read in edited tree
            trees = Phylo.parse(tree_dict[branch]["tree_file_edit"], "nexus")
            # There should be only 1 tree
            for t in trees:
                tree_dict[branch]["tree"] = t
                tree_dict[branch]["tree"].ladderize(reverse=False)
                break

            # Rename sample names back to with dashes
            for c in tree_dict[branch]["tree"].find_clades():
                if c.name in tree_dict[branch]["sample_rename"]:
                    orig_name = c.name
                    c.name = tree_dict[branch]["sample_rename"][c.name]
                    print("Rename:", orig_name, c.name)
                    
                # Strip the date suffix
                if c.name:
                    c.name = "_".join(c.name.split("_")[0:-1])

Branch: 1.ORI
<generator object parse at 0x7fee40ac24f8>
Rename: GCA_000324805.2_EV76_CN_genomic_90 GCA_000324805.2_EV76-CN_genomic_90
Rename: GCA_000986995.1_YPES001_SEQ_2_ASM_1_genomic_22 GCA_000986995.1_YPES001-SEQ-2-ASM-1_genomic_22
Branch: 1.IN
<generator object parse at 0x7fee7fda9480>
Branch: 1.ANT
<generator object parse at 0x7fee40ac2b10>
Branch: 1.PRE
<generator object parse at 0x7fee40ac2930>
Branch: 2.MED
<generator object parse at 0x7fee40ac29a8>
Rename: GCA_001617735.1_Yersinia_pestis_M_1763_genomic_16 GCA_001617735.1_Yersinia_pestis_M-1763_genomic_16
Rename: GCA_002412305.1_Y.pestis_A_1809_genomic_38 GCA_002412305.1_Y.pestis_A-1809_genomic_38
Rename: GCA_001617785.1_Yersinia_pestis_M_549_genomic_41 GCA_001617785.1_Yersinia_pestis_M-549_genomic_41
Rename: GCA_001617725.1_Yersinia_pestis_M_1484_genomic_26 GCA_001617725.1_Yersinia_pestis_M-1484_genomic_26
Rename: GCA_001617715.1_Yersinia_pestis_M_519_genomic_51 GCA_001617715.1_Yersinia_pestis_M-519_genomic_51
Rename: GCA_00

## Add Tree Data to Dataframe

In [9]:
# Initialize new columns

metadata_df["node_type"] = [NO_DATA_CHAR] * len(metadata_df)
metadata_df["branch_length"] = [NO_DATA_CHAR] * len(metadata_df)
metadata_df["rate"] = [NO_DATA_CHAR] * len(metadata_df)
metadata_df["rate_subs"] = [NO_DATA_CHAR] * len(metadata_df)
metadata_df["timetree_num_date_confidence"] = [[0,0]] * len(metadata_df)

init_data = {col: [NO_DATA_CHAR] for col in metadata_df.columns}

for branch in BRANCH_LIST:
    tree = tree_dict[branch]["tree"]

    df = copy.deepcopy(metadata_df[metadata_df["branch_minor"].isin(BRANCH_LIST[branch])])
    
    node_i = 0
    
    # Get comment headers for df
    root_comment_dict = parse_comment(tree.root.comment)
    for parameter in root_comment_dict:
        df[parameter] = [NO_DATA_CHAR] * len(df)
    
    for c in tree.find_clades():

        # Rename internal nodes
        if not c.name:
            c.name = "NODE{}".format(node_i)
            node_i += 1

        # Initialize metadata for internal nodes
        if "NODE" in c.name:
            init_data["sample"] = c.name
            data_row = pd.DataFrame(init_data)
            data_row.set_index("sample", inplace=True)
            df = df.append(data_row)
            
        # Set node type
        if "NODE" in c.name:
            node_type = "internal"                    
        else:
            node_type = "terminal"
           
        # Parse comments
        comment_dict = parse_comment(c.comment)
        for parameter,val in comment_dict.items():
            df.at[c.name, parameter] = val
            
        # Get calendar date for node
        calendar_date = mrsd_dict[branch] - float(comment_dict["height"])
        df.at[c.name, "timetree_num_date"] = calendar_date
        
        # Rate
        if df["rate"][c.name] == NO_DATA_CHAR:
            df.at[c.name, "rate"] = 0
        else:
            df.at[c.name, "rate"] = float(df["rate"][c.name])
            
        rate = df["rate"][c.name]
        rate_subs = rate * SEQ_LEN
        df.at[c.name, "rate_subs"] = rate_subs
        
        
        # Convert
        height = comment_dict["height"]
        df.at[c.name, "height"] = float(height)
        if "height_95%_HPD" in comment_dict:
            height_95_hpd = comment_dict["height_95%_HPD"]
            height_95_hpd_list = [float(h) for h in height_95_hpd.strip("{}").split(",")]
            calendar_95_hpd = [mrsd_dict[branch] - h for h in height_95_hpd_list]
            df.at[c.name, "timetree_num_date_confidence"] = calendar_95_hpd
        
        # Add data
        df.at[c.name, "node_type"] = node_type
        df.at[c.name, "branch_length"] = c.branch_length

        
        
    # Replace NAs for inconsistent parameters
    df.fillna(NO_DATA_CHAR, inplace=True)
    tree_dict[branch]["df"] = copy.deepcopy(df)

    
    display(tree_dict[branch]["df"])


,strain,date,date_bp,country,province,country_lat,country_lon,province_lat,province_lon,biovar,...,height_median,height_range,height,timetree_num_date,length_range,rate_95%_HPD,length_95%_HPD,rate_range,rate_median,length_median
sample,,,,,,,,,,,,,,,,,,,,,
Reference,CO92,1992,-29,United States of America,Colorado,39.7837,-100.446,38.7252,-105.608,Orientalis,...,24.0,"{23.99999999999997,24.00000000000003}",24.000000,1992.000000,"{0.7875393122285885,42.88322836127834}","{5.927714716500102E-8,9.377226168027601E-7}","{1.8427926537330457,31.191320981608243}","{4.558602518220217E-8,5.030931524742551E-6}",2.406871030661851E-7,15.627963378354789
GCA_000834775.1_ASM83477v1_genomic,Dodson,1967.0,-54,United States of America,Arizona,39.7837,-100.446,34.3953,-111.763,Orientalis,...,49.0,"{48.99999999999994,49.00000000000006}",49.000000,1967.000000,"{0.2888043881030171,31.897590286999048}","{3.797620557463332E-8,8.218425612961558E-7}","{0.5471598315006432,15.847060277531043}","{3.429741540262673E-8,5.587063400379781E-6}",2.11059520244398E-7,6.8709840507756965
GCA_000834335.1_ASM83433v1_genomic,Shasta,1954.0,-67,United States of America,California,39.7837,-100.446,36.7015,-118.756,Orientalis,...,62.0,"{61.99999999999994,62.00000000000006}",62.000000,1954.000000,"{0.05748774760200348,28.285740849649073}","{9.133766746749261E-9,6.323190957865774E-7}","{0.17495681504591687,12.737728729624394}","{8.854592434294595E-9,1.0406855638781851E-5}",1.3435424636585818E-7,4.22066337192107
GCA_000169635.1_ASM16963v1_genomic,MG05-1020,2005.0,-16,Madagascar,NA,-18.925,46.4416,NA,NA,Orientalis,...,11.0,"{10.999999999999886,11.000000000000114}",11.000000,2005.000000,"{8.752165669176431,295.5000109399901}","{1.233569124508418E-8,2.3007345037878172E-7}","{10.589101358759361,123.010848907607}","{2.4954871920903633E-9,6.269225969490349E-7}",9.170997173986732E-8,31.207977460374813
GCA_000170275.1_ASM17027v1_genomic,F1991016,1991.0,-30,China,Yunnan,35.0001,105,25,102,Orientalis,...,25.0,"{24.999999999999915,25.000000000000114}",25.000000,1991.000000,"{0.2631175955287688,44.1379271393413}","{1.335420651988494E-8,1.744799312573441E-7}","{4.1125205412269175,25.461413283591654}","{7.906603889093561E-9,8.136140381553714E-6}",6.45564613655224E-8,14.682564786518267
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NODE111,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,11.539676109019013,"{7.792388509136714,21.612246615199865}",11.758628,2004.241372,"{7.716029389008838E-4,11.312580561638354}","{1.9839302934854463E-9,3.12708553465188E-7}","{7.716029389008838E-4,4.39721893027677}","{1.3911910216568945E-9,4.122322400208003E-6}",5.489378888559102E-8,0.876279012156477
NODE112,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,8.700173180241187,"{6.223285280518212,20.760408533631022}",9.000250,2006.999750,"{0.014623758088442074,13.240021205452297}","{3.99933677937127E-9,3.777893049181355E-7}","{0.04462381365286738,6.681793251825638}","{1.2441548987194403E-10,2.184978370128529E-6}",8.569340550062568E-8,2.584203274463891
NODE113,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,9.90892202420423,"{6.250713139639117,20.59096663766985}",10.153429,2005.846571,"{0.013053801168137369,14.049681722628108}","{4.988541760516177E-9,5.009242745920154E-7}","{0.0364229231711537,5.210916537824495}","{3.2679056126377386E-9,3.673718936276794E-6}",1.0063395444008382E-7,1.7175801021600883


,strain,date,date_bp,country,province,country_lat,country_lon,province_lat,province_lon,biovar,...,height_median,height_range,height,timetree_num_date,length_range,rate_95%_HPD,length_95%_HPD,rate_range,rate_median,length_median
sample,,,,,,,,,,,,,,,,,,,,,
GCA_000324785.1_ASM32478v1_genomic,E1979001,1979.0,-42,China,Yunnan,35.0001,105,25,102,Antiqua,...,29.0,"{28.999999999999773,29.000000000000227}",29.000000,1979.000000,"{0.07053329864158187,96.89515848563791}","{7.673453580186803E-9,5.883054151776795E-7}","{0.11419523126047082,23.71869144866433}","{5.227670710953615E-9,1.5277265670001108E-5}",1.3741552572303324E-7,7.1001361305315065
GCA_000022845.1_ASM2284v1_genomic,Z176003,1976.0,-45,China,Tibet,35.0001,105,29.8556,90.875,Antiqua,...,32.0,"{31.999999999999886,32.00000000000023}",32.000000,1976.000000,"{0.020933807108228564,82.79777607969515}","{1.1789626162994611E-8,1.0496928932779852E-6}","{0.18117432834666403,14.605601141678335}","{8.12966038651375E-9,2.9386383189083592E-5}",1.9978077504526392E-7,4.725233739427562
GCA_000022825.1_ASM2282v1_genomic,D182038,1982.0,-39,China,Yunnan,35.0001,105,25,102,Antiqua,...,26.0,"{25.999999999999773,26.000000000000227}",26.000000,1982.000000,"{0.08937106070665024,589.3209625195739}","{2.2540148977519486E-7,4.594696163978095E-6}","{1.8639875641122075,40.638623151994295}","{5.257222416398857E-9,2.1249306444951562E-4}",1.6451429285041112E-6,12.837618164448656
GCA_000022805.1_ASM2280v1_genomic,D106004,2006.0,-15,China,Yunnan,35.0001,105,25,102,Antiqua,...,2.0,"{1.9999999999997726,2.0000000000002274}",2.000000,2006.000000,"{30.90870302205029,1344.3719576228896}","{4.553195303942278E-11,2.9080100342260826E-7}","{55.042636129386665,337.8708114185053}","{4.553195303942278E-11,5.602655078605247E-7}",1.492939174392166E-7,114.2128756875396
GCA_000325465.1_ASM32546v1_genomic,YN472,1957.0,-64,China,Yunnan,35.0001,105,25,102,Antiqua,...,51.0,"{50.99999999999977,51.00000000000023}",51.000000,1957.000000,"{0.05002176568445549,304.876066390452}","{1.9837563597011857E-9,7.956755664617774E-7}","{0.05002176568445549,30.862122041125957}","{7.82487835707705E-10,4.322756703533504E-5}",1.589424805067659E-7,8.982024941876588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NODE33,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,33.80092577690954,"{20.148590558614472,97.53173478912154}",35.202750,1972.797250,"{0.34331453942335344,972.9166238156288}","{2.3193055667342354E-10,1.4345574786636522E-7}","{1.177952560432324,56.86233651128582}","{1.3282405812591905E-11,2.577025781297195E-6}",3.7071149270117E-8,23.05313608692641
NODE34,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,25.746913555282497,"{19.051458135432398,63.73450808106165}",26.478588,1981.521412,"{0.014421434439192637,231.53258856629301}","{7.751474461993943E-9,7.165890167490827E-7}","{0.2112549484949433,21.25863387518203}","{1.0784411783761654E-9,2.388662444696541E-5}",1.3931835019056373E-7,7.151702259012737
NODE35,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,20.295580766107143,"{12.357882961727043,39.148926327889114}",20.784843,1987.215157,"{0.01584406505847369,33.50327285175207}","{1.9573317023209576E-9,3.797525216697401E-7}","{0.14705247869108717,13.001737647906406}","{1.7676031108587427E-9,5.767390045817478E-5}",7.493142257342362E-8,4.903135571992461


,strain,date,date_bp,country,province,country_lat,country_lon,province_lat,province_lon,biovar,...,height_median,height_range,height,timetree_num_date,length_range,rate_95%_HPD,length_95%_HPD,rate_median,length_median,rate_range
sample,,,,,,,,,,,,,,,,,,,,,
GCA_000169615.1_ASM16961v1_genomic,UG05-0454,2004.0,-17,Uganda,NA,1.53336,32.2167,NA,NA,Antiqua,...,NA,NA,0.000000,2004.000000,"{39.03071480967925,81.72889396786833}","{3.567705616814925E-8,9.526254372431744E-8}","{40.54166853057723,59.71031826868307}",6.322643060893905E-8,51.09844040271891,"{2.0559437911431462E-8,1.397949778207173E-7}"
GCA_003798225.1_ASM379822v1_genomic,FDAARGOS_601,1954.0,-67,Democratic Republic of the Congo,NA,-2.98143,23.8223,NA,NA,Antiqua,...,50.0,"{49.99999999999994,50.00000000000006}",50.000000,1954.000000,"{1.2575546398352344E-4,34.25157217725058}","{2.5174294873000518E-8,8.401882379686717E-7}","{1.2575546398352344E-4,7.203493943401831}",6.64555669779107E-8,1.3068457691474151,"{2.0559437911431462E-8,7.176853010058904E-4}"
GCA_000835005.1_ASM83500v1_genomic,Nairobi,1985.0,-36,Kenya,Nairobi,1.44197,38.4314,-1.30317,36.8261,Antiqua,...,19.0,"{18.999999999999943,19.000000000000114}",19.000000,1985.000000,"{95.63366685339598,661.730948495061}","{3.567705616814925E-8,9.526254372431744E-8}","{129.95605585874753,340.16541877082847}",6.322643060893905E-8,217.60128389929963,"{2.0559437911431462E-8,1.397949778207173E-7}"
GCA_000013825.1_ASM1382v1_genomic,Antiqua,1965.0,-56,Democratic Republic of the Congo,NA,-2.98143,23.8223,NA,NA,Antiqua,...,39.0,"{38.99999999999994,39.000000000000114}",39.000000,1965.000000,"{0.030714809679253108,44.948193480799574}","{3.0555436269571344E-8,1.0432566356133833E-7}","{1.1947284446334194,20.880629956997204}",6.397711690575176E-8,12.28405650191602,"{2.0559437911431462E-8,5.321978221012509E-5}"
NODE0,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,236.60128389929963,"{114.633666853396,680.730948495061}",245.089772,1758.910228,NA,NA,NA,NA,NA,NA
NODE1,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,52.64374069994703,"{50.00012575546398,84.25157217725058}",53.672275,1950.327725,"{64.61279327382427,607.4965736954912}","{3.567705616814925E-8,9.526254372431744E-8}","{97.79366988602827,302.68409735227175}",6.322643060893905E-8,183.33856834908784,"{2.0559437911431462E-8,1.397949778207173E-7}"
NODE2,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,49.47620066420954,"{39.03071480967924,67.3053400861503}",49.030949,1954.969051,"{1.272980906961152E-4,22.731515487253063}","{2.9144286845089494E-8,6.755254810454324E-7}","{1.272980906961152E-4,10.315737315490011}",6.878079611290572E-8,2.2648298874066555,"{2.6842462089992865E-8,5.6892025236036795E-5}"


,strain,date,date_bp,country,province,country_lat,country_lon,province_lat,province_lon,biovar,...,height_median,height_range,height,timetree_num_date,length_range,rate_95%_HPD,length_95%_HPD,rate_range,rate_median,length_median
sample,,,,,,,,,,,,,,,,,,,,,
SAMEA5818830,STN021,[1485:1635],[-536:-386],Switzerland,Nidwalden,46.8133,8.44495,46.9428,8.41198,Second Pandemic,...,207.5,"{207.49999999999977,207.50000000000034}",207.500000,1560.000000,"{0.010327534017591233,19.090134869009262}","{1.6235429369193934E-12,1.2578028656475436E-7}","{0.012754330225533295,5.920170144662478}","{1.6235429369193934E-12,1.401848539870756E-6}",1.5942421850309658E-8,1.8197050435199174
SAMEA5818829,STN020,[1485:1635],[-536:-386],Switzerland,Nidwalden,46.8133,8.44495,46.9428,8.41198,Second Pandemic,...,207.5,"{207.49999999999977,207.50000000000034}",207.500000,1560.000000,"{0.0015626184052450753,24.330597985222425}","{1.340154478399539E-12,1.2497432122346923E-7}","{0.0015626184052450753,8.878479685942125}","{1.340154478399539E-12,1.9033534499916213E-6}",1.413333091052279E-8,2.2318468564765794
SAMEA5818828,STN019,[1485:1635],[-536:-386],Switzerland,Nidwalden,46.8133,8.44495,46.9428,8.41198,Second Pandemic,...,207.5,"{207.49999999999977,207.50000000000034}",207.500000,1560.000000,"{0.0015626184052450753,28.570033822193693}","{3.313640785811537E-13,1.245574570871356E-7}","{0.0015626184052450753,8.65980649625834}","{3.313640785811537E-13,2.172775088641939E-6}",1.4815708488368307E-8,2.1664770690543094
SAMEA5818826,STN014,[1485:1635],[-536:-386],Switzerland,Nidwalden,46.8133,8.44495,46.9428,8.41198,Second Pandemic,...,207.5,"{207.49999999999972,207.50000000000034}",207.500000,1560.000000,"{0.0015903754216708421,23.930180106234275}","{2.0913896342255515E-13,1.1795152529087725E-7}","{0.0015903754216708421,8.94392395949501}","{2.0913896342255515E-13,1.4229134790094017E-6}",1.5360190349716956E-8,2.254896682876307
SAMEA5818825,STN013,[1485:1635],[-536:-386],Switzerland,Nidwalden,46.8133,8.44495,46.9428,8.41198,Second Pandemic,...,207.5,"{207.49999999999977,207.50000000000034}",207.500000,1560.000000,"{0.010327534017591233,19.090134869009262}","{2.2950766842256673E-10,4.4984495805689245E-7}","{0.012754330225533295,5.920170144662478}","{2.2950766842256673E-10,2.8087035111913184E-5}",6.39376828984195E-8,1.8197050435199174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NODE34,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,134.5484568852686,"{118.11524821327026,173.8999772889183}",135.787581,1631.712419,"{0.08320637260970898,59.13853089027987}","{2.42939921729192E-10,1.6223052548935136E-7}","{0.08320637260970898,29.511320888204608}","{2.42939921729192E-10,4.9416665766820575E-6}",2.5809587707835425E-8,9.265643801492814
NODE35,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,73.21035924501427,"{49.63839941071973,159.60320394232866}",77.111273,1690.388727,"{2.113034506645647,133.4401755213923}","{2.4356687053733948E-9,3.48694265698337E-8}","{27.855229821069116,105.8639145711434}","{8.04284556323758E-10,6.036173699790764E-7}",1.3813542524478606E-8,72.05454223510617
NODE36,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,57.23371610050458,"{47.45677351807575,116.11511361433179}",58.961997,1708.538003,"{0.1887228617228942,91.98953366074421}","{8.746518287619847E-10,1.66446581870181E-7}","{0.24779294774728555,49.103833006269156}","{8.746518287619847E-10,3.028608620173103E-6}",2.852315482568535E-8,13.983590806062857


,strain,date,date_bp,country,province,country_lat,country_lon,province_lat,province_lon,biovar,...,height_median,height_range,height,timetree_num_date,length_range,rate_95%_HPD,length_95%_HPD,rate_range,rate_median,length_median
sample,,,,,,,,,,,,,,,,,,,,,
GCA_009909635.1_ASM990963v1_genomic,9_10,1923.0,-98,Russia,Rostov Oblast,64.6863,97.7453,47.6222,40.7958,Medievalis,...,95.0,"{94.99999999999977,95.00000000000045}",95.000000,1923.000000,"{0.5053148565945662,119.80747133076287}","{1.0954638933725613E-8,5.118150074045258E-7}","{0.6513012276527519,39.90188082750021}","{7.794809022625615E-9,3.107279640981197E-6}",1.0405292553722971E-7,16.259614371142362
GCA_009296005.1_ASM929600v1_genomic,C-25,1953.0,-68,Russia,Chechnya,64.6863,97.7453,43.3976,45.6985,Medievalis,...,65.0,"{64.99999999999966,65.00000000000045}",65.000000,1953.000000,"{0.020546491332211758,65.41284700148023}","{3.234057962463953E-10,2.623366951363394E-7}","{0.020546491332211758,32.7063058406128}","{3.234057962463953E-10,6.613383681972808E-6}",2.461109520538659E-8,9.53451342922753
GCA_008630485.1_ASM863048v1_genomic,C-742,1997.0,-24,Russia,Kabardino-Balkaria,64.6863,97.7453,43.4428,43.4205,Medievalis,...,21.0,"{20.999999999999545,21.00000000000057}",21.000000,1997.000000,"{0.10813926548556552,18.3055943899154}","{5.244187850052271E-8,2.3672739203135E-6}","{0.10813926548556552,8.51519468100092}","{1.7062054223203027E-8,1.2616686100482554E-5}",4.2245833947465984E-7,3.1018499439360347
GCA_008630435.1_ASM863043v1_genomic,C-719,1996.0,-25,Russia,Karachay-Cherkessia,64.6863,97.7453,43.7368,41.7268,Medievalis,...,22.0,"{21.999999999999773,22.000000000000455}",22.000000,1996.000000,"{5.824406736948738,57.74892193772499}","{2.6858159939169463E-7,1.1726725352472136E-6}","{11.518137909263658,40.67058314820686}","{2.0176377010359712E-7,1.8361633003867908E-6}",6.319948300750766E-7,22.952158422812452
GCA_008630395.1_ASM863039v1_genomic,C-528,1984.0,-37,Russia,Republic of Dagestan,64.6863,97.7453,43.0883,47.1499,Medievalis,...,34.00000000000003,"{33.99999999999932,34.00000000000057}",34.000000,1984.000000,"{0.09697458176516704,27.018513542580862}","{3.24729784984071E-8,1.6790623586932283E-6}","{0.09697458176516704,14.621983081230162}","{2.33989032397258E-8,2.5101491269209814E-5}",3.004645590964938E-7,5.876811746053114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NODE110,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,28.825262275444317,"{21.56819556725742,42.723163257443275}",29.122886,1988.877114,"{0.14148238897868026,24.51765676513019}","{4.8985635588274965E-9,4.258832808359966E-7}","{0.42101810216856705,14.974098815115639}","{2.273742820162396E-9,4.538665968634544E-6}",8.247106017834924E-8,7.946356134673016
NODE111,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,12.21616464793054,"{9.001194215922482,33.04616727478188}",13.507166,2004.492834,"{0.0303484820581037,27.729865699927224}","{2.5268931171405116E-10,2.3643508836198407E-7}","{0.13217685921050304,19.70793161462254}","{2.5268931171405116E-10,5.45437686190429E-6}",2.8536790528243043E-8,8.230788425258048
NODE112,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,26.294758630682878,"{21.69079031783332,36.15971932511246}",26.784019,1991.215981,"{0.00362986670153731,19.745955426203363}","{2.0678704070748174E-11,2.411263730761986E-7}","{0.00362986670153731,8.147182447605648}","{2.0678704070748174E-11,1.5206306828007214E-6}",1.6367720968612385E-8,1.868720101063424


...

,strain,date,date_bp,country,province,country_lat,country_lon,province_lat,province_lon,biovar,...,height_median,height_range,height,timetree_num_date,length_range,rate_95%_HPD,length_95%_HPD,rate_range,rate_median,length_median
sample,,,,,,,,,,,,,,,,,,,,,
GCA_000325305.1_ASM32530v1_genomic,MGJZ9,1998.0,-23,Mongolia,Govi-Altai,46.825,103.85,45.3463,95.9326,Antiqua,...,19.0,"{18.999999999999886,19.000000000000114}",19.000000,1998.000000,"{1.162431120427243E-4,48.32480418253621}","{1.6294101752494803E-12,1.504102045000881E-7}","{1.162431120427243E-4,16.363747934126067}","{1.6294101752494803E-12,4.5199726772005674E-5}",5.1282300344550075E-9,3.725846546440838
GCA_000325285.1_ASM32528v1_genomic,MGJZ7,1997.0,-24,Mongolia,Dornogovi,46.825,103.85,44.418,109.778,Antiqua,...,20.0,"{19.999999999999886,20.000000000000114}",20.000000,1997.000000,"{0.020092700462139845,37.26253639964513}","{8.584183302457259E-9,1.4467667419088765E-6}","{0.02848444078836465,16.92268695314634}","{8.584183302457259E-9,8.817666594060108E-5}",1.8626320818152352E-7,5.883265201838054
GCA_000325265.1_ASM32526v1_genomic,MGJZ6,1997.0,-24,Mongolia,Dornogovi,46.825,103.85,44.418,109.778,Antiqua,...,20.0,"{19.999999999999886,20.000000000000114}",20.000000,1997.000000,"{0.020092700462139845,37.26253639964513}","{4.003483826943674E-10,6.200612802722695E-7}","{0.02848444078836465,16.92268695314634}","{4.003483826943674E-10,1.592166505601846E-5}",6.831322249360667E-8,5.883265201838054
GCA_000325245.1_ASM32524v1_genomic,MGJZ3,1980.0,-41,Mongolia,Govi-Altai,46.825,103.85,45.3463,95.9326,Antiqua,...,37.0,"{36.999999999999886,37.00000000000006}",37.000000,1980.000000,"{0.007944043154338942,136.76551340010172}","{1.0400836500820097E-8,1.383397831676533E-5}","{0.008640140537472973,9.916333898128599}","{1.0400836500820097E-8,4.5728155734759513E-4}",1.5584596565498745E-6,2.320164604609456
GCA_000325205.1_ASM32520v1_genomic,MGJZ11,2000.0,-21,Mongolia,Bayan-Ölgii,46.825,103.85,48.547,89.8549,Antiqua,...,17.0,"{16.999999999999886,17.000000000000114}",17.000000,2000.000000,"{2.0001162431120427,50.32480418253621}","{9.357959599314852E-9,5.054231789899088E-7}","{2.0001162431120427,18.363747934126067}","{6.4963414771820826E-9,1.2967433406375449E-6}",1.422693662758114E-7,5.725846546440838
GCA_000324325.1_ASM32432v1_genomic,CMCC71001,1961.0,-60,China,Gansu,35.0001,105,38,102,Antiqua,...,56.0,"{55.999999999999886,56.00000000000006}",56.000000,1961.000000,"{0.005354855812349513,479.3500964922347}","{9.225797518126407E-12,1.2203965153272926E-6}","{0.005354855812349513,65.55316567766563}","{9.225797518126407E-12,2.1333307815013134E-4}",1.0437840058360544E-7,10.330964856050898
GCA_000323765.1_ASM32376v1_genomic,C1976001,1976.0,-45,China,Gansu,35.0001,105,38,102,Antiqua,...,41.0,"{40.99999999999977,41.000000000000114}",41.000000,1976.000000,"{0.009732556474276066,320.0205261068583}","{1.5007668107099268E-11,1.1622428080510877E-6}","{0.009732556474276066,31.636556511175655}","{1.5007668107099268E-11,1.3197032905035508E-4}",1.0825606409944168E-7,7.955572591547838
GCA_000323525.1_ASM32352v1_genomic,71021,1989.0,-32,China,Gansu,35.0001,105,38,102,Antiqua,...,28.0,"{27.999999999999773,28.000000000000114}",28.000000,1989.000000,"{13.009732556474276,523.2139090009956}","{4.212255019656925E-9,1.021068272980644E-7}","{13.009732556474276,44.340463675491335}","{4.834894303383254E-12,2.4124553125504745E-7}",3.973823623184727E-8,20.95557259154784
GCA_000323505.1_ASM32350v1_genomic,7,2004.0,-17,China,Qinghai,35.0001,105,35.4071,95.9521,Antiqua,...,13.0,"{12.999999999999886,13.000000000000114}",13.000000,2004.000000,"{0.4898256559139842,741.2135422419725}","{7.494652904783449E-12,8.296737333677962E-8}","{29.359551195911124,136.97550545051715}","{4.314041033499446E-12,1.3204184258465739E-5}",4.0672911170022175E-8,59.93626543975124


,strain,date,date_bp,country,province,country_lat,country_lon,province_lat,province_lon,biovar,...,height_median,height_range,height,timetree_num_date,length_range,rate_95%_HPD,length_95%_HPD,rate_range,rate_median,length_median
sample,,,,,,,,,,,,,,,,,,,,,
GCA_009669545.1_ASM966954v1_genomic,42126,2006.0,-15,China,Xinjiang,35.0001,105,42.4805,85.4633,Antiqua,...,13.0,"{12.999999999999545,13.000000000000682}",13.000000,2006.000000,"{1.0206388938108741,44.364996483883004}","{7.403347397235594E-11,4.883393805859067E-8}","{1.2490761851000833,20.21173229257289}","{2.121834316064548E-11,2.8752960742270225E-7}",9.808403647353713E-9,7.364671294340331
GCA_009669555.1_ASM966955v1_genomic,42123,2005.0,-16,China,Xinjiang,35.0001,105,42.4805,85.4633,Antiqua,...,14.0,"{13.999999999999545,14.000000000000682}",14.000000,2005.000000,"{0.020638893810874137,42.868983599858275}","{3.4771974801962574E-10,1.03294106073432E-7}","{0.2932097593989127,21.459448347835945}","{3.4771974801962574E-10,1.550099538844396E-5}",2.3970193373355508E-8,7.268687707804949
GCA_009669565.1_ASM966956v1_genomic,42118,2005.0,-16,China,Xinjiang,35.0001,105,42.4805,85.4633,Antiqua,...,14.0,"{13.999999999999545,14.000000000000455}",14.000000,2005.000000,"{0.3185777149361346,39.70550740233376}","{1.4905064121497649E-9,1.461593938615892E-7}","{1.2115293305370365,23.59132959368069}","{1.400591641814841E-9,1.4824145942528173E-6}",3.8674789299045825E-8,8.180878702163444
GCA_009669605.1_ASM966960v1_genomic,42117,2005.0,-16,China,Xinjiang,35.0001,105,42.4805,85.4633,Antiqua,...,14.0,"{13.999999999999318,14.000000000000682}",14.000000,2005.000000,"{0.09048420987639005,43.98145880570236}","{6.808621579893027E-10,1.0001987033446645E-7}","{0.3347414141589784,21.758767400096367}","{2.6022236857973256E-10,7.114360759296812E-7}",2.3609942910990694E-8,7.398892203579367
GCA_009669625.1_ASM966962v1_genomic,42116,2005.0,-16,China,Xinjiang,35.0001,105,42.4805,85.4633,Antiqua,...,14.0,"{13.999999999999545,14.000000000000682}",14.000000,2005.000000,"{5.550818242649854,25.65051195184857}","{9.389715449019844E-12,3.583188598679891E-8}","{10.000189261625376,16.09249720241116}","{2.578415156580025E-12,1.1047127254992546E-7}",8.44059496277305E-9,11.442123860131774
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NODE97,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,22.43351351123448,"{20.004225343760027,33.77546728890735}",23.759111,1995.240889,"{0.009745717525195374,16.555662244077862}","{6.733701947703063E-10,9.632978966630426E-8}","{0.009745717525195374,13.231829146587305}","{6.733701947703063E-10,2.5838420378069403E-7}",1.268152143049444E-8,2.539556643790309
NODE98,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,19.525892303125886,"{18.002568969646916,36.50760361032269}",20.596841,1998.403159,"{0.01982270055254176,37.492315036100024}","{1.846941549126493E-10,8.915235160152371E-8}","{0.01982270055254176,14.928018981759717}","{1.846941549126493E-10,5.450124044279921E-7}",1.3387109358127652E-8,3.030234834490342
NODE99,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,24.17587937636864,"{18.86392719358321,37.853792831312944}",25.274371,1993.725629,"{0.04525845118746474,22.498689050761044}","{1.915765415780841E-9,4.5888054416319054E-8}","{0.04525845118746474,16.29129973579562}","{1.915765415780841E-9,5.4216561103893366E-8}",1.3815894917483899E-8,2.894269856826483


,strain,date,date_bp,country,province,country_lat,country_lon,province_lat,province_lon,biovar,...,height_median,height_range,height,timetree_num_date,length_range,rate_95%_HPD,length_95%_HPD,rate_median,length_median,rate_range
sample,,,,,,,,,,,,,,,,,,,,,
SAMEA5661390,UNT004,[419:537],[-1602:-1484],Germany,Bavaria,51.0834,10.4234,48.9468,11.4039,Antiqua,...,287.0,"{286.9999999999998,287.0000000000002}",287.000000,478.000000,"{0.09618773214117482,65.69464454954039}","{5.749435332333931E-8,4.997353851545937E-6}","{0.1486988685201709,19.907877533714498}",7.769361422601801E-7,5.264482559681795,"{4.583372482588842E-8,4.5639102653091446E-5}"
SAMEA5661389,UNT003,[433:599],[-1588:-1422],Germany,Bavaria,51.0834,10.4234,48.9468,11.4039,Antiqua,...,249.0,"{248.99999999999977,249.00000000000023}",249.000000,516.000000,"{1.2655654393146847,92.65047188645087}","{8.532895370102424E-14,1.589406737583535E-8}","{4.537624887280515,40.85420808591613}",6.843648009000825E-10,17.53214359603612,"{8.532895370102424E-14,1.5052230240630748E-7}"
SAMEA5661385,PET004,[434:600],[-1587:-1421],Germany,Bavaria,51.0834,10.4234,48.9468,11.4039,Antiqua,...,248.0,"{247.9999999999999,248.0000000000001}",248.000000,517.000000,"{0.19394660782788264,121.83290647924622}","{1.9680091798712527E-8,2.5275371837605134E-6}","{0.20518025754898872,52.511333183995816}",3.3720674437223856E-7,12.714583183804734,"{1.9680091798712527E-8,2.0070540947175287E-5}"
SAMEA5661384,VAL001,[432:610],[-1589:-1411],Spain,Valencia Community,39.3261,-4.83798,39.682,-0.765441,Antiqua,...,244.0,"{243.99999999999977,244.00000000000023}",244.000000,521.000000,"{1.8564017701386888,293.56883772305935}","{3.5148173064671363E-9,3.6386567422467276E-8}","{46.57475866073054,135.4631995370337}",1.683806477701757E-8,76.01748269006436,"{1.4972611156125737E-9,5.821484575588239E-7}"
SAMEA5661372,LSD020,[650:880],[-1371:-1141],France,Centre-Loire Valley,46.6034,1.88833,47.549,1.73241,Antiqua,...,NA,NA,0.000000,765.000000,"{0.09449997404021829,261.79155616345594}","{4.193713286299026E-9,2.8786787287404325E-7}","{0.8779015479523723,166.01406761309266}",4.090283535604485E-8,49.799053839949394,"{3.2609422628707773E-9,2.4213154550370125E-5}"
SAMEA5661367,LSD001,[650:880],[-1371:-1141],France,Centre-Loire Valley,46.6034,1.88833,47.549,1.73241,Antiqua,...,NA,NA,0.000000,765.000000,"{0.09449997404021829,261.79155616345594}","{6.251215471055465E-9,4.180723404978003E-7}","{0.8779015479523723,166.01406761309266}",5.656814084554631E-8,49.799053839949394,"{5.888192894089655E-9,2.4213154550370125E-5}"
SAMEA5661365,EDI004,[500:650],[-1521:-1371],England,East of England,52.531,-1.26491,52.22,0.487578,Antiqua,...,190.0,"{189.99999999999966,190.00000000000023}",190.000000,575.000000,"{0.21166249115407254,371.8688799409464}","{1.0295500121048277E-10,1.0475103093436425E-7}","{0.3429755491576998,66.89264892177721}",1.4743229439811216E-8,27.055720807342595,"{1.0295500121048277E-10,2.9397297953444343E-6}"
SAMEA5661364,EDI003,[500:650],[-1521:-1371],England,East of England,52.531,-1.26491,52.22,0.487578,Antiqua,...,190.0,"{189.99999999999966,190.00000000000023}",190.000000,575.000000,"{0.21166249115407254,165.63170787821696}","{1.1136279590388429E-8,5.934060639723203E-7}","{0.3429755491576998,55.54076467772276}",1.0745075548517368E-7,23.838301456070056,"{1.1136279590388429E-8,1.322899932614607E-5}"
SAMEA5661363,EDI001,[474:637],[-1547:-1384],England,East of England,52.531,-1.26491,52.22,0.487578,Antiqua,...,209.5,"{209.49999999999977,209.50000000000023}",209.500000,555.500000,"{5.656192944343275E-4,138.04593848975412}","{7.242961669621194E-14,6.424251239986978E-8}","{5.656192944343275E-4,38.63322683531038}",1.6008156925623966E-9,8.58184741017162,"{7.242961669621194E-14,1.397341822947193E-5}"


,strain,date,date_bp,country,province,country_lat,country_lon,province_lat,province_lon,biovar,...,height_median,height_range,height,timetree_num_date,length_range,rate_95%_HPD,length_95%_HPD,rate_range,rate_median,length_median
sample,,,,,,,,,,,,,,,,,,,,,
GCA_008630455.1_ASM863045v1_genomic,C-826,2011.0,-10,Russia,Republic of Dagestan,64.6863,97.7453,43.0883,47.1499,Caucasica,...,3.0,"{2.999999999999659,3.0000000000002274}",3,2011,"{1.4273937684314273,44.852179285076105}","{2.6097465824417483E-9,6.321194324773121E-8}","{23.0251026957521,42.265688865474765}","{4.4171353372083285E-10,5.349289484410833E-7}",2.614544478805884E-8,29.66182276861643
GCA_008630415.1_ASM863041v1_genomic,C-672,1988.0,-33,Russia,Republic of Dagestan,64.6863,97.7453,43.0883,47.1499,Caucasica,...,26.0,"{25.99999999999966,26.000000000000227}",26,1988,"{0.0999317572162326,23.69872820107333}","{7.249413352729134E-8,1.6183423240693155E-6}","{0.3849389753981036,12.650602031963281}","{5.802709582733856E-8,1.694733787991764E-5}",3.4131437267050303E-7,6.86367764896692
GCA_006546745.1_ASM654674v1_genomic,I-3269,1990.0,-31,Mongolia,Bayan-Ölgii,46.825,103.85,48.547,89.8549,Ulegeica,...,24.0,"{23.999999999999773,24.000000000000227}",24,1990,"{3.366442708043376,76.18426571714122}","{7.048475896756691E-8,2.3683698306483924E-7}","{19.53422000238826,32.922449442003376}","{4.681916477954927E-8,1.6808983934032898E-6}",1.4883208765293748E-7,24.410008270255773
GCA_006546725.1_ASM654672v1_genomic,I-3196,1986.0,-35,Mongolia,Govi-Altai,46.825,103.85,45.3463,95.9326,Ulegeica,...,28.0,"{27.999999999999773,28.000000000000227}",28,1986,"{0.04291121951107968,86.74750643904615}","{2.0895228099157223E-8,1.4969767541491584E-7}","{14.286108804231745,34.74944354281658}","{8.823724288792062E-9,3.795225847417498E-5}",7.512125355179427E-8,21.449633713234775
GCA_006546665.1_ASM654666v1_genomic,I-3193,1987.0,-34,Mongolia,Khovd,46.825,103.85,48.0552,91.2673,Ulegeica,...,27.0,"{26.999999999999773,27.000000000000227}",27,1987,"{0.3664427080433761,48.908055263553806}","{6.793126768110123E-8,2.757304038486022E-7}","{13.10086710239635,26.103285610587037}","{3.599308325113401E-8,8.873055986291867E-6}",1.5246054297245548E-7,17.273348532222585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NODE79,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,30.29976917492388,"{14.955244317050486,47.9927667968382}",29.8877,1984.11,"{0.00156249379370621,35.41337498703514}","{4.164124831771047E-11,2.672094384408922E-7}","{0.00156249379370621,14.696016550014907}","{4.164124831771047E-11,9.014611514970941E-6}",2.3909672920519704E-8,4.639186880136492
NODE80,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,21.952472290115224,"{13.365015229852695,48.355588003336834}",22.4691,1991.53,"{0.036866763887143605,39.920587628336904}","{1.0488273462693892E-8,1.1994835376745965E-6}","{0.05819741258296318,17.349332562662205}","{1.0488273462693892E-8,2.8837272616003555E-5}",1.772764553018911E-7,6.4773424370369455
NODE81,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,27.70599160170255,"{14.64755034917772,44.49255052369415}",27.6095,1986.39,"{0.0011262655275672273,30.54458913823302}","{1.4255370435288464E-10,3.0569162872441313E-7}","{0.0011262655275672273,13.299656375518357}","{1.4255370435288464E-10,1.0531650076286934E-5}",3.013465807629524E-8,3.4804997915285334


,strain,date,date_bp,country,province,country_lat,country_lon,province_lat,province_lon,biovar,...,height_median,height_range,height,timetree_num_date,length_range,rate_95%_HPD,length_95%_HPD,rate_median,length_median,rate_range
sample,,,,,,,,,,,,,,,,,,,,,
SAMEA104233048,Post6,[-2007:-1882],[-4028:-3903],Germany,Bavaria,51.0834,10.4234,48.9468,11.4039,Bronze Age,...,NA,NA,258.500000,-1944.500000,"{0.26137597812328295,592.5612425268795}","{5.899275191756727E-9,6.628500059441837E-8}","{10.368349896758673,434.47299698367635}",1.1056113813575873E-8,279.7068901438097,"{1.7638360457688605E-9,5.162673494796717E-6}"
SAMEA104233049,1343UnTal85,[-2396:-2148],[-4417:-4169],Germany,Bavaria,51.0834,10.4234,48.9468,11.4039,Bronze Age,...,586.0,"{585.9999999999998,586.0000000000002}",586.000000,-2272.000000,"{0.019873221045259015,438.6664342603435}","{7.06178400605268E-9,4.901162112344934E-7}","{0.0365347912403422,286.9071437031581}",1.3738619000466807E-8,198.52007956847336,"{7.06178400605268E-9,2.6725273156174455E-4}"
SAMEA104233047,KunilaII,[-2574:-2340],[-4595:-4361],Estonia,Pärnu maakond,58.7524,25.3319,58.3195,24.3026,Bronze Age,...,771.0,"{770.9999999999998,771.0000000000002}",771.000000,-2457.000000,"{15.516357303293034,579.4696931574999}","{5.914885313936088E-9,1.664021441329337E-8}","{212.14118263263276,322.9907129516921}",1.0634238460469731E-8,247.73285748072288,"{3.553982434339283E-11,3.1476740218664647E-7}"
SAMEA104233046,Gyvakarai1,[-2621:-2472],[-4642:-4493],Lithuania,Panevezys County,55.35,23.75,55.9156,25.0312,Bronze Age,...,860.5,"{860.4999999999998,860.5000000000002}",860.500000,-2546.500000,"{124.26191842019148,794.1352557281409}","{5.954375754189968E-9,1.8556102757533683E-8}","{137.0963765275194,333.62429982253525}",1.084629076731244E-8,234.42400727706092,"{1.681707147922761E-11,5.143391916110305E-8}"
SAMEA3541827,RISE509,[-2876:-2677],[-4897:-4698],Russia,Krasnoyarsk Krai,64.6863,97.7453,63.3234,97.098,Bronze Age,...,1090.5,"{1090.4999999999998,1090.5000000000002}",1090.500000,-2776.500000,"{0.009520840034610956,631.4029061404672}","{9.43993461581082E-12,3.855538614541085E-6}","{0.009520840034610956,195.1582901865247}",2.7835411003429926E-7,35.8665897510715,"{9.43993461581082E-12,2.4017124916796598E-4}"
SAMEA3541826,RISE505,[-1746:-1626],[-3767:-3647],Russia,Altai Krai,64.6863,97.7453,52.6932,82.6931,Bronze Age,...,NA,NA,0.000000,-1686.000000,"{258.7613759781233,851.0612425268795}","{4.175019264222575E-8,1.2997492607400134E-7}","{268.8683498967587,692.9729969836764}",6.594314426461917E-8,538.2068901438097,"{3.7354052936568944E-8,1.7900722210928818E-7}"
SAMEA6812877,GLZ002,[-2569:-2467],[-4590:-4488],Russia,Irkutsk Oblast,64.6863,97.7453,56.637,104.719,Bronze Age,...,832.0,"{831.9999999999998,832.0000000000002}",832.000000,-2518.000000,"{148.50405789372724,216.9712877158113}","{6.458078866578565E-12,1.4976644269696073E-8}","{148.50405789372724,168.64795492796281}",9.858374234245279E-9,152.3608818717251,"{6.458078866578565E-12,5.143391916110305E-8}"
SAMEA6812876,GLZ001,[-2838:-2495],[-4859:-4516],Russia,Irkutsk Oblast,64.6863,97.7453,56.637,104.719,Bronze Age,...,980.5,"{980.4999999999998,980.5000000000002}",980.500000,-2666.500000,"{0.004057893727235751,68.47128771581129}","{6.634839520346167E-10,1.8886659867901838E-6}","{0.004057893727235751,20.147954927962815}",1.4360584565765109E-7,3.8608818717250983,"{6.634839520346167E-10,1.9629055065027202E-4}"
NODE0,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,1143.3570853433978,"{1090.5095208400346,1721.9029061404672}",1166.091245,-2852.091245,NA,NA,NA,NA,NA,NA


## Manual Editing

In [10]:
for branch in tree_dict:
    
    
    for rec in tree_dict[branch]["df"].iterrows():
        # Rate change
        ...
        
    display(tree_dict[branch]["df"])

,strain,date,date_bp,country,province,country_lat,country_lon,province_lat,province_lon,biovar,...,height_median,height_range,height,timetree_num_date,length_range,rate_95%_HPD,length_95%_HPD,rate_range,rate_median,length_median
sample,,,,,,,,,,,,,,,,,,,,,
Reference,CO92,1992,-29,United States of America,Colorado,39.7837,-100.446,38.7252,-105.608,Orientalis,...,24.0,"{23.99999999999997,24.00000000000003}",24.000000,1992.000000,"{0.7875393122285885,42.88322836127834}","{5.927714716500102E-8,9.377226168027601E-7}","{1.8427926537330457,31.191320981608243}","{4.558602518220217E-8,5.030931524742551E-6}",2.406871030661851E-7,15.627963378354789
GCA_000834775.1_ASM83477v1_genomic,Dodson,1967.0,-54,United States of America,Arizona,39.7837,-100.446,34.3953,-111.763,Orientalis,...,49.0,"{48.99999999999994,49.00000000000006}",49.000000,1967.000000,"{0.2888043881030171,31.897590286999048}","{3.797620557463332E-8,8.218425612961558E-7}","{0.5471598315006432,15.847060277531043}","{3.429741540262673E-8,5.587063400379781E-6}",2.11059520244398E-7,6.8709840507756965
GCA_000834335.1_ASM83433v1_genomic,Shasta,1954.0,-67,United States of America,California,39.7837,-100.446,36.7015,-118.756,Orientalis,...,62.0,"{61.99999999999994,62.00000000000006}",62.000000,1954.000000,"{0.05748774760200348,28.285740849649073}","{9.133766746749261E-9,6.323190957865774E-7}","{0.17495681504591687,12.737728729624394}","{8.854592434294595E-9,1.0406855638781851E-5}",1.3435424636585818E-7,4.22066337192107
GCA_000169635.1_ASM16963v1_genomic,MG05-1020,2005.0,-16,Madagascar,NA,-18.925,46.4416,NA,NA,Orientalis,...,11.0,"{10.999999999999886,11.000000000000114}",11.000000,2005.000000,"{8.752165669176431,295.5000109399901}","{1.233569124508418E-8,2.3007345037878172E-7}","{10.589101358759361,123.010848907607}","{2.4954871920903633E-9,6.269225969490349E-7}",9.170997173986732E-8,31.207977460374813
GCA_000170275.1_ASM17027v1_genomic,F1991016,1991.0,-30,China,Yunnan,35.0001,105,25,102,Orientalis,...,25.0,"{24.999999999999915,25.000000000000114}",25.000000,1991.000000,"{0.2631175955287688,44.1379271393413}","{1.335420651988494E-8,1.744799312573441E-7}","{4.1125205412269175,25.461413283591654}","{7.906603889093561E-9,8.136140381553714E-6}",6.45564613655224E-8,14.682564786518267
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NODE111,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,11.539676109019013,"{7.792388509136714,21.612246615199865}",11.758628,2004.241372,"{7.716029389008838E-4,11.312580561638354}","{1.9839302934854463E-9,3.12708553465188E-7}","{7.716029389008838E-4,4.39721893027677}","{1.3911910216568945E-9,4.122322400208003E-6}",5.489378888559102E-8,0.876279012156477
NODE112,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,8.700173180241187,"{6.223285280518212,20.760408533631022}",9.000250,2006.999750,"{0.014623758088442074,13.240021205452297}","{3.99933677937127E-9,3.777893049181355E-7}","{0.04462381365286738,6.681793251825638}","{1.2441548987194403E-10,2.184978370128529E-6}",8.569340550062568E-8,2.584203274463891
NODE113,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,9.90892202420423,"{6.250713139639117,20.59096663766985}",10.153429,2005.846571,"{0.013053801168137369,14.049681722628108}","{4.988541760516177E-9,5.009242745920154E-7}","{0.0364229231711537,5.210916537824495}","{3.2679056126377386E-9,3.673718936276794E-6}",1.0063395444008382E-7,1.7175801021600883


,strain,date,date_bp,country,province,country_lat,country_lon,province_lat,province_lon,biovar,...,height_median,height_range,height,timetree_num_date,length_range,rate_95%_HPD,length_95%_HPD,rate_range,rate_median,length_median
sample,,,,,,,,,,,,,,,,,,,,,
GCA_000324785.1_ASM32478v1_genomic,E1979001,1979.0,-42,China,Yunnan,35.0001,105,25,102,Antiqua,...,29.0,"{28.999999999999773,29.000000000000227}",29.000000,1979.000000,"{0.07053329864158187,96.89515848563791}","{7.673453580186803E-9,5.883054151776795E-7}","{0.11419523126047082,23.71869144866433}","{5.227670710953615E-9,1.5277265670001108E-5}",1.3741552572303324E-7,7.1001361305315065
GCA_000022845.1_ASM2284v1_genomic,Z176003,1976.0,-45,China,Tibet,35.0001,105,29.8556,90.875,Antiqua,...,32.0,"{31.999999999999886,32.00000000000023}",32.000000,1976.000000,"{0.020933807108228564,82.79777607969515}","{1.1789626162994611E-8,1.0496928932779852E-6}","{0.18117432834666403,14.605601141678335}","{8.12966038651375E-9,2.9386383189083592E-5}",1.9978077504526392E-7,4.725233739427562
GCA_000022825.1_ASM2282v1_genomic,D182038,1982.0,-39,China,Yunnan,35.0001,105,25,102,Antiqua,...,26.0,"{25.999999999999773,26.000000000000227}",26.000000,1982.000000,"{0.08937106070665024,589.3209625195739}","{2.2540148977519486E-7,4.594696163978095E-6}","{1.8639875641122075,40.638623151994295}","{5.257222416398857E-9,2.1249306444951562E-4}",1.6451429285041112E-6,12.837618164448656
GCA_000022805.1_ASM2280v1_genomic,D106004,2006.0,-15,China,Yunnan,35.0001,105,25,102,Antiqua,...,2.0,"{1.9999999999997726,2.0000000000002274}",2.000000,2006.000000,"{30.90870302205029,1344.3719576228896}","{4.553195303942278E-11,2.9080100342260826E-7}","{55.042636129386665,337.8708114185053}","{4.553195303942278E-11,5.602655078605247E-7}",1.492939174392166E-7,114.2128756875396
GCA_000325465.1_ASM32546v1_genomic,YN472,1957.0,-64,China,Yunnan,35.0001,105,25,102,Antiqua,...,51.0,"{50.99999999999977,51.00000000000023}",51.000000,1957.000000,"{0.05002176568445549,304.876066390452}","{1.9837563597011857E-9,7.956755664617774E-7}","{0.05002176568445549,30.862122041125957}","{7.82487835707705E-10,4.322756703533504E-5}",1.589424805067659E-7,8.982024941876588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NODE33,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,33.80092577690954,"{20.148590558614472,97.53173478912154}",35.202750,1972.797250,"{0.34331453942335344,972.9166238156288}","{2.3193055667342354E-10,1.4345574786636522E-7}","{1.177952560432324,56.86233651128582}","{1.3282405812591905E-11,2.577025781297195E-6}",3.7071149270117E-8,23.05313608692641
NODE34,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,25.746913555282497,"{19.051458135432398,63.73450808106165}",26.478588,1981.521412,"{0.014421434439192637,231.53258856629301}","{7.751474461993943E-9,7.165890167490827E-7}","{0.2112549484949433,21.25863387518203}","{1.0784411783761654E-9,2.388662444696541E-5}",1.3931835019056373E-7,7.151702259012737
NODE35,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,20.295580766107143,"{12.357882961727043,39.148926327889114}",20.784843,1987.215157,"{0.01584406505847369,33.50327285175207}","{1.9573317023209576E-9,3.797525216697401E-7}","{0.14705247869108717,13.001737647906406}","{1.7676031108587427E-9,5.767390045817478E-5}",7.493142257342362E-8,4.903135571992461


,strain,date,date_bp,country,province,country_lat,country_lon,province_lat,province_lon,biovar,...,height_median,height_range,height,timetree_num_date,length_range,rate_95%_HPD,length_95%_HPD,rate_median,length_median,rate_range
sample,,,,,,,,,,,,,,,,,,,,,
GCA_000169615.1_ASM16961v1_genomic,UG05-0454,2004.0,-17,Uganda,NA,1.53336,32.2167,NA,NA,Antiqua,...,NA,NA,0.000000,2004.000000,"{39.03071480967925,81.72889396786833}","{3.567705616814925E-8,9.526254372431744E-8}","{40.54166853057723,59.71031826868307}",6.322643060893905E-8,51.09844040271891,"{2.0559437911431462E-8,1.397949778207173E-7}"
GCA_003798225.1_ASM379822v1_genomic,FDAARGOS_601,1954.0,-67,Democratic Republic of the Congo,NA,-2.98143,23.8223,NA,NA,Antiqua,...,50.0,"{49.99999999999994,50.00000000000006}",50.000000,1954.000000,"{1.2575546398352344E-4,34.25157217725058}","{2.5174294873000518E-8,8.401882379686717E-7}","{1.2575546398352344E-4,7.203493943401831}",6.64555669779107E-8,1.3068457691474151,"{2.0559437911431462E-8,7.176853010058904E-4}"
GCA_000835005.1_ASM83500v1_genomic,Nairobi,1985.0,-36,Kenya,Nairobi,1.44197,38.4314,-1.30317,36.8261,Antiqua,...,19.0,"{18.999999999999943,19.000000000000114}",19.000000,1985.000000,"{95.63366685339598,661.730948495061}","{3.567705616814925E-8,9.526254372431744E-8}","{129.95605585874753,340.16541877082847}",6.322643060893905E-8,217.60128389929963,"{2.0559437911431462E-8,1.397949778207173E-7}"
GCA_000013825.1_ASM1382v1_genomic,Antiqua,1965.0,-56,Democratic Republic of the Congo,NA,-2.98143,23.8223,NA,NA,Antiqua,...,39.0,"{38.99999999999994,39.000000000000114}",39.000000,1965.000000,"{0.030714809679253108,44.948193480799574}","{3.0555436269571344E-8,1.0432566356133833E-7}","{1.1947284446334194,20.880629956997204}",6.397711690575176E-8,12.28405650191602,"{2.0559437911431462E-8,5.321978221012509E-5}"
NODE0,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,236.60128389929963,"{114.633666853396,680.730948495061}",245.089772,1758.910228,NA,NA,NA,NA,NA,NA
NODE1,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,52.64374069994703,"{50.00012575546398,84.25157217725058}",53.672275,1950.327725,"{64.61279327382427,607.4965736954912}","{3.567705616814925E-8,9.526254372431744E-8}","{97.79366988602827,302.68409735227175}",6.322643060893905E-8,183.33856834908784,"{2.0559437911431462E-8,1.397949778207173E-7}"
NODE2,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,49.47620066420954,"{39.03071480967924,67.3053400861503}",49.030949,1954.969051,"{1.272980906961152E-4,22.731515487253063}","{2.9144286845089494E-8,6.755254810454324E-7}","{1.272980906961152E-4,10.315737315490011}",6.878079611290572E-8,2.2648298874066555,"{2.6842462089992865E-8,5.6892025236036795E-5}"


,strain,date,date_bp,country,province,country_lat,country_lon,province_lat,province_lon,biovar,...,height_median,height_range,height,timetree_num_date,length_range,rate_95%_HPD,length_95%_HPD,rate_range,rate_median,length_median
sample,,,,,,,,,,,,,,,,,,,,,
SAMEA5818830,STN021,[1485:1635],[-536:-386],Switzerland,Nidwalden,46.8133,8.44495,46.9428,8.41198,Second Pandemic,...,207.5,"{207.49999999999977,207.50000000000034}",207.500000,1560.000000,"{0.010327534017591233,19.090134869009262}","{1.6235429369193934E-12,1.2578028656475436E-7}","{0.012754330225533295,5.920170144662478}","{1.6235429369193934E-12,1.401848539870756E-6}",1.5942421850309658E-8,1.8197050435199174
SAMEA5818829,STN020,[1485:1635],[-536:-386],Switzerland,Nidwalden,46.8133,8.44495,46.9428,8.41198,Second Pandemic,...,207.5,"{207.49999999999977,207.50000000000034}",207.500000,1560.000000,"{0.0015626184052450753,24.330597985222425}","{1.340154478399539E-12,1.2497432122346923E-7}","{0.0015626184052450753,8.878479685942125}","{1.340154478399539E-12,1.9033534499916213E-6}",1.413333091052279E-8,2.2318468564765794
SAMEA5818828,STN019,[1485:1635],[-536:-386],Switzerland,Nidwalden,46.8133,8.44495,46.9428,8.41198,Second Pandemic,...,207.5,"{207.49999999999977,207.50000000000034}",207.500000,1560.000000,"{0.0015626184052450753,28.570033822193693}","{3.313640785811537E-13,1.245574570871356E-7}","{0.0015626184052450753,8.65980649625834}","{3.313640785811537E-13,2.172775088641939E-6}",1.4815708488368307E-8,2.1664770690543094
SAMEA5818826,STN014,[1485:1635],[-536:-386],Switzerland,Nidwalden,46.8133,8.44495,46.9428,8.41198,Second Pandemic,...,207.5,"{207.49999999999972,207.50000000000034}",207.500000,1560.000000,"{0.0015903754216708421,23.930180106234275}","{2.0913896342255515E-13,1.1795152529087725E-7}","{0.0015903754216708421,8.94392395949501}","{2.0913896342255515E-13,1.4229134790094017E-6}",1.5360190349716956E-8,2.254896682876307
SAMEA5818825,STN013,[1485:1635],[-536:-386],Switzerland,Nidwalden,46.8133,8.44495,46.9428,8.41198,Second Pandemic,...,207.5,"{207.49999999999977,207.50000000000034}",207.500000,1560.000000,"{0.010327534017591233,19.090134869009262}","{2.2950766842256673E-10,4.4984495805689245E-7}","{0.012754330225533295,5.920170144662478}","{2.2950766842256673E-10,2.8087035111913184E-5}",6.39376828984195E-8,1.8197050435199174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NODE34,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,134.5484568852686,"{118.11524821327026,173.8999772889183}",135.787581,1631.712419,"{0.08320637260970898,59.13853089027987}","{2.42939921729192E-10,1.6223052548935136E-7}","{0.08320637260970898,29.511320888204608}","{2.42939921729192E-10,4.9416665766820575E-6}",2.5809587707835425E-8,9.265643801492814
NODE35,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,73.21035924501427,"{49.63839941071973,159.60320394232866}",77.111273,1690.388727,"{2.113034506645647,133.4401755213923}","{2.4356687053733948E-9,3.48694265698337E-8}","{27.855229821069116,105.8639145711434}","{8.04284556323758E-10,6.036173699790764E-7}",1.3813542524478606E-8,72.05454223510617
NODE36,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,57.23371610050458,"{47.45677351807575,116.11511361433179}",58.961997,1708.538003,"{0.1887228617228942,91.98953366074421}","{8.746518287619847E-10,1.66446581870181E-7}","{0.24779294774728555,49.103833006269156}","{8.746518287619847E-10,3.028608620173103E-6}",2.852315482568535E-8,13.983590806062857


,strain,date,date_bp,country,province,country_lat,country_lon,province_lat,province_lon,biovar,...,height_median,height_range,height,timetree_num_date,length_range,rate_95%_HPD,length_95%_HPD,rate_range,rate_median,length_median
sample,,,,,,,,,,,,,,,,,,,,,
GCA_009909635.1_ASM990963v1_genomic,9_10,1923.0,-98,Russia,Rostov Oblast,64.6863,97.7453,47.6222,40.7958,Medievalis,...,95.0,"{94.99999999999977,95.00000000000045}",95.000000,1923.000000,"{0.5053148565945662,119.80747133076287}","{1.0954638933725613E-8,5.118150074045258E-7}","{0.6513012276527519,39.90188082750021}","{7.794809022625615E-9,3.107279640981197E-6}",1.0405292553722971E-7,16.259614371142362
GCA_009296005.1_ASM929600v1_genomic,C-25,1953.0,-68,Russia,Chechnya,64.6863,97.7453,43.3976,45.6985,Medievalis,...,65.0,"{64.99999999999966,65.00000000000045}",65.000000,1953.000000,"{0.020546491332211758,65.41284700148023}","{3.234057962463953E-10,2.623366951363394E-7}","{0.020546491332211758,32.7063058406128}","{3.234057962463953E-10,6.613383681972808E-6}",2.461109520538659E-8,9.53451342922753
GCA_008630485.1_ASM863048v1_genomic,C-742,1997.0,-24,Russia,Kabardino-Balkaria,64.6863,97.7453,43.4428,43.4205,Medievalis,...,21.0,"{20.999999999999545,21.00000000000057}",21.000000,1997.000000,"{0.10813926548556552,18.3055943899154}","{5.244187850052271E-8,2.3672739203135E-6}","{0.10813926548556552,8.51519468100092}","{1.7062054223203027E-8,1.2616686100482554E-5}",4.2245833947465984E-7,3.1018499439360347
GCA_008630435.1_ASM863043v1_genomic,C-719,1996.0,-25,Russia,Karachay-Cherkessia,64.6863,97.7453,43.7368,41.7268,Medievalis,...,22.0,"{21.999999999999773,22.000000000000455}",22.000000,1996.000000,"{5.824406736948738,57.74892193772499}","{2.6858159939169463E-7,1.1726725352472136E-6}","{11.518137909263658,40.67058314820686}","{2.0176377010359712E-7,1.8361633003867908E-6}",6.319948300750766E-7,22.952158422812452
GCA_008630395.1_ASM863039v1_genomic,C-528,1984.0,-37,Russia,Republic of Dagestan,64.6863,97.7453,43.0883,47.1499,Medievalis,...,34.00000000000003,"{33.99999999999932,34.00000000000057}",34.000000,1984.000000,"{0.09697458176516704,27.018513542580862}","{3.24729784984071E-8,1.6790623586932283E-6}","{0.09697458176516704,14.621983081230162}","{2.33989032397258E-8,2.5101491269209814E-5}",3.004645590964938E-7,5.876811746053114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NODE110,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,28.825262275444317,"{21.56819556725742,42.723163257443275}",29.122886,1988.877114,"{0.14148238897868026,24.51765676513019}","{4.8985635588274965E-9,4.258832808359966E-7}","{0.42101810216856705,14.974098815115639}","{2.273742820162396E-9,4.538665968634544E-6}",8.247106017834924E-8,7.946356134673016
NODE111,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,12.21616464793054,"{9.001194215922482,33.04616727478188}",13.507166,2004.492834,"{0.0303484820581037,27.729865699927224}","{2.5268931171405116E-10,2.3643508836198407E-7}","{0.13217685921050304,19.70793161462254}","{2.5268931171405116E-10,5.45437686190429E-6}",2.8536790528243043E-8,8.230788425258048
NODE112,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,26.294758630682878,"{21.69079031783332,36.15971932511246}",26.784019,1991.215981,"{0.00362986670153731,19.745955426203363}","{2.0678704070748174E-11,2.411263730761986E-7}","{0.00362986670153731,8.147182447605648}","{2.0678704070748174E-11,1.5206306828007214E-6}",1.6367720968612385E-8,1.868720101063424


...

,strain,date,date_bp,country,province,country_lat,country_lon,province_lat,province_lon,biovar,...,height_median,height_range,height,timetree_num_date,length_range,rate_95%_HPD,length_95%_HPD,rate_range,rate_median,length_median
sample,,,,,,,,,,,,,,,,,,,,,
GCA_000325305.1_ASM32530v1_genomic,MGJZ9,1998.0,-23,Mongolia,Govi-Altai,46.825,103.85,45.3463,95.9326,Antiqua,...,19.0,"{18.999999999999886,19.000000000000114}",19.000000,1998.000000,"{1.162431120427243E-4,48.32480418253621}","{1.6294101752494803E-12,1.504102045000881E-7}","{1.162431120427243E-4,16.363747934126067}","{1.6294101752494803E-12,4.5199726772005674E-5}",5.1282300344550075E-9,3.725846546440838
GCA_000325285.1_ASM32528v1_genomic,MGJZ7,1997.0,-24,Mongolia,Dornogovi,46.825,103.85,44.418,109.778,Antiqua,...,20.0,"{19.999999999999886,20.000000000000114}",20.000000,1997.000000,"{0.020092700462139845,37.26253639964513}","{8.584183302457259E-9,1.4467667419088765E-6}","{0.02848444078836465,16.92268695314634}","{8.584183302457259E-9,8.817666594060108E-5}",1.8626320818152352E-7,5.883265201838054
GCA_000325265.1_ASM32526v1_genomic,MGJZ6,1997.0,-24,Mongolia,Dornogovi,46.825,103.85,44.418,109.778,Antiqua,...,20.0,"{19.999999999999886,20.000000000000114}",20.000000,1997.000000,"{0.020092700462139845,37.26253639964513}","{4.003483826943674E-10,6.200612802722695E-7}","{0.02848444078836465,16.92268695314634}","{4.003483826943674E-10,1.592166505601846E-5}",6.831322249360667E-8,5.883265201838054
GCA_000325245.1_ASM32524v1_genomic,MGJZ3,1980.0,-41,Mongolia,Govi-Altai,46.825,103.85,45.3463,95.9326,Antiqua,...,37.0,"{36.999999999999886,37.00000000000006}",37.000000,1980.000000,"{0.007944043154338942,136.76551340010172}","{1.0400836500820097E-8,1.383397831676533E-5}","{0.008640140537472973,9.916333898128599}","{1.0400836500820097E-8,4.5728155734759513E-4}",1.5584596565498745E-6,2.320164604609456
GCA_000325205.1_ASM32520v1_genomic,MGJZ11,2000.0,-21,Mongolia,Bayan-Ölgii,46.825,103.85,48.547,89.8549,Antiqua,...,17.0,"{16.999999999999886,17.000000000000114}",17.000000,2000.000000,"{2.0001162431120427,50.32480418253621}","{9.357959599314852E-9,5.054231789899088E-7}","{2.0001162431120427,18.363747934126067}","{6.4963414771820826E-9,1.2967433406375449E-6}",1.422693662758114E-7,5.725846546440838
GCA_000324325.1_ASM32432v1_genomic,CMCC71001,1961.0,-60,China,Gansu,35.0001,105,38,102,Antiqua,...,56.0,"{55.999999999999886,56.00000000000006}",56.000000,1961.000000,"{0.005354855812349513,479.3500964922347}","{9.225797518126407E-12,1.2203965153272926E-6}","{0.005354855812349513,65.55316567766563}","{9.225797518126407E-12,2.1333307815013134E-4}",1.0437840058360544E-7,10.330964856050898
GCA_000323765.1_ASM32376v1_genomic,C1976001,1976.0,-45,China,Gansu,35.0001,105,38,102,Antiqua,...,41.0,"{40.99999999999977,41.000000000000114}",41.000000,1976.000000,"{0.009732556474276066,320.0205261068583}","{1.5007668107099268E-11,1.1622428080510877E-6}","{0.009732556474276066,31.636556511175655}","{1.5007668107099268E-11,1.3197032905035508E-4}",1.0825606409944168E-7,7.955572591547838
GCA_000323525.1_ASM32352v1_genomic,71021,1989.0,-32,China,Gansu,35.0001,105,38,102,Antiqua,...,28.0,"{27.999999999999773,28.000000000000114}",28.000000,1989.000000,"{13.009732556474276,523.2139090009956}","{4.212255019656925E-9,1.021068272980644E-7}","{13.009732556474276,44.340463675491335}","{4.834894303383254E-12,2.4124553125504745E-7}",3.973823623184727E-8,20.95557259154784
GCA_000323505.1_ASM32350v1_genomic,7,2004.0,-17,China,Qinghai,35.0001,105,35.4071,95.9521,Antiqua,...,13.0,"{12.999999999999886,13.000000000000114}",13.000000,2004.000000,"{0.4898256559139842,741.2135422419725}","{7.494652904783449E-12,8.296737333677962E-8}","{29.359551195911124,136.97550545051715}","{4.314041033499446E-12,1.3204184258465739E-5}",4.0672911170022175E-8,59.93626543975124


,strain,date,date_bp,country,province,country_lat,country_lon,province_lat,province_lon,biovar,...,height_median,height_range,height,timetree_num_date,length_range,rate_95%_HPD,length_95%_HPD,rate_range,rate_median,length_median
sample,,,,,,,,,,,,,,,,,,,,,
GCA_009669545.1_ASM966954v1_genomic,42126,2006.0,-15,China,Xinjiang,35.0001,105,42.4805,85.4633,Antiqua,...,13.0,"{12.999999999999545,13.000000000000682}",13.000000,2006.000000,"{1.0206388938108741,44.364996483883004}","{7.403347397235594E-11,4.883393805859067E-8}","{1.2490761851000833,20.21173229257289}","{2.121834316064548E-11,2.8752960742270225E-7}",9.808403647353713E-9,7.364671294340331
GCA_009669555.1_ASM966955v1_genomic,42123,2005.0,-16,China,Xinjiang,35.0001,105,42.4805,85.4633,Antiqua,...,14.0,"{13.999999999999545,14.000000000000682}",14.000000,2005.000000,"{0.020638893810874137,42.868983599858275}","{3.4771974801962574E-10,1.03294106073432E-7}","{0.2932097593989127,21.459448347835945}","{3.4771974801962574E-10,1.550099538844396E-5}",2.3970193373355508E-8,7.268687707804949
GCA_009669565.1_ASM966956v1_genomic,42118,2005.0,-16,China,Xinjiang,35.0001,105,42.4805,85.4633,Antiqua,...,14.0,"{13.999999999999545,14.000000000000455}",14.000000,2005.000000,"{0.3185777149361346,39.70550740233376}","{1.4905064121497649E-9,1.461593938615892E-7}","{1.2115293305370365,23.59132959368069}","{1.400591641814841E-9,1.4824145942528173E-6}",3.8674789299045825E-8,8.180878702163444
GCA_009669605.1_ASM966960v1_genomic,42117,2005.0,-16,China,Xinjiang,35.0001,105,42.4805,85.4633,Antiqua,...,14.0,"{13.999999999999318,14.000000000000682}",14.000000,2005.000000,"{0.09048420987639005,43.98145880570236}","{6.808621579893027E-10,1.0001987033446645E-7}","{0.3347414141589784,21.758767400096367}","{2.6022236857973256E-10,7.114360759296812E-7}",2.3609942910990694E-8,7.398892203579367
GCA_009669625.1_ASM966962v1_genomic,42116,2005.0,-16,China,Xinjiang,35.0001,105,42.4805,85.4633,Antiqua,...,14.0,"{13.999999999999545,14.000000000000682}",14.000000,2005.000000,"{5.550818242649854,25.65051195184857}","{9.389715449019844E-12,3.583188598679891E-8}","{10.000189261625376,16.09249720241116}","{2.578415156580025E-12,1.1047127254992546E-7}",8.44059496277305E-9,11.442123860131774
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NODE97,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,22.43351351123448,"{20.004225343760027,33.77546728890735}",23.759111,1995.240889,"{0.009745717525195374,16.555662244077862}","{6.733701947703063E-10,9.632978966630426E-8}","{0.009745717525195374,13.231829146587305}","{6.733701947703063E-10,2.5838420378069403E-7}",1.268152143049444E-8,2.539556643790309
NODE98,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,19.525892303125886,"{18.002568969646916,36.50760361032269}",20.596841,1998.403159,"{0.01982270055254176,37.492315036100024}","{1.846941549126493E-10,8.915235160152371E-8}","{0.01982270055254176,14.928018981759717}","{1.846941549126493E-10,5.450124044279921E-7}",1.3387109358127652E-8,3.030234834490342
NODE99,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,24.17587937636864,"{18.86392719358321,37.853792831312944}",25.274371,1993.725629,"{0.04525845118746474,22.498689050761044}","{1.915765415780841E-9,4.5888054416319054E-8}","{0.04525845118746474,16.29129973579562}","{1.915765415780841E-9,5.4216561103893366E-8}",1.3815894917483899E-8,2.894269856826483


,strain,date,date_bp,country,province,country_lat,country_lon,province_lat,province_lon,biovar,...,height_median,height_range,height,timetree_num_date,length_range,rate_95%_HPD,length_95%_HPD,rate_median,length_median,rate_range
sample,,,,,,,,,,,,,,,,,,,,,
SAMEA5661390,UNT004,[419:537],[-1602:-1484],Germany,Bavaria,51.0834,10.4234,48.9468,11.4039,Antiqua,...,287.0,"{286.9999999999998,287.0000000000002}",287.000000,478.000000,"{0.09618773214117482,65.69464454954039}","{5.749435332333931E-8,4.997353851545937E-6}","{0.1486988685201709,19.907877533714498}",7.769361422601801E-7,5.264482559681795,"{4.583372482588842E-8,4.5639102653091446E-5}"
SAMEA5661389,UNT003,[433:599],[-1588:-1422],Germany,Bavaria,51.0834,10.4234,48.9468,11.4039,Antiqua,...,249.0,"{248.99999999999977,249.00000000000023}",249.000000,516.000000,"{1.2655654393146847,92.65047188645087}","{8.532895370102424E-14,1.589406737583535E-8}","{4.537624887280515,40.85420808591613}",6.843648009000825E-10,17.53214359603612,"{8.532895370102424E-14,1.5052230240630748E-7}"
SAMEA5661385,PET004,[434:600],[-1587:-1421],Germany,Bavaria,51.0834,10.4234,48.9468,11.4039,Antiqua,...,248.0,"{247.9999999999999,248.0000000000001}",248.000000,517.000000,"{0.19394660782788264,121.83290647924622}","{1.9680091798712527E-8,2.5275371837605134E-6}","{0.20518025754898872,52.511333183995816}",3.3720674437223856E-7,12.714583183804734,"{1.9680091798712527E-8,2.0070540947175287E-5}"
SAMEA5661384,VAL001,[432:610],[-1589:-1411],Spain,Valencia Community,39.3261,-4.83798,39.682,-0.765441,Antiqua,...,244.0,"{243.99999999999977,244.00000000000023}",244.000000,521.000000,"{1.8564017701386888,293.56883772305935}","{3.5148173064671363E-9,3.6386567422467276E-8}","{46.57475866073054,135.4631995370337}",1.683806477701757E-8,76.01748269006436,"{1.4972611156125737E-9,5.821484575588239E-7}"
SAMEA5661372,LSD020,[650:880],[-1371:-1141],France,Centre-Loire Valley,46.6034,1.88833,47.549,1.73241,Antiqua,...,NA,NA,0.000000,765.000000,"{0.09449997404021829,261.79155616345594}","{4.193713286299026E-9,2.8786787287404325E-7}","{0.8779015479523723,166.01406761309266}",4.090283535604485E-8,49.799053839949394,"{3.2609422628707773E-9,2.4213154550370125E-5}"
SAMEA5661367,LSD001,[650:880],[-1371:-1141],France,Centre-Loire Valley,46.6034,1.88833,47.549,1.73241,Antiqua,...,NA,NA,0.000000,765.000000,"{0.09449997404021829,261.79155616345594}","{6.251215471055465E-9,4.180723404978003E-7}","{0.8779015479523723,166.01406761309266}",5.656814084554631E-8,49.799053839949394,"{5.888192894089655E-9,2.4213154550370125E-5}"
SAMEA5661365,EDI004,[500:650],[-1521:-1371],England,East of England,52.531,-1.26491,52.22,0.487578,Antiqua,...,190.0,"{189.99999999999966,190.00000000000023}",190.000000,575.000000,"{0.21166249115407254,371.8688799409464}","{1.0295500121048277E-10,1.0475103093436425E-7}","{0.3429755491576998,66.89264892177721}",1.4743229439811216E-8,27.055720807342595,"{1.0295500121048277E-10,2.9397297953444343E-6}"
SAMEA5661364,EDI003,[500:650],[-1521:-1371],England,East of England,52.531,-1.26491,52.22,0.487578,Antiqua,...,190.0,"{189.99999999999966,190.00000000000023}",190.000000,575.000000,"{0.21166249115407254,165.63170787821696}","{1.1136279590388429E-8,5.934060639723203E-7}","{0.3429755491576998,55.54076467772276}",1.0745075548517368E-7,23.838301456070056,"{1.1136279590388429E-8,1.322899932614607E-5}"
SAMEA5661363,EDI001,[474:637],[-1547:-1384],England,East of England,52.531,-1.26491,52.22,0.487578,Antiqua,...,209.5,"{209.49999999999977,209.50000000000023}",209.500000,555.500000,"{5.656192944343275E-4,138.04593848975412}","{7.242961669621194E-14,6.424251239986978E-8}","{5.656192944343275E-4,38.63322683531038}",1.6008156925623966E-9,8.58184741017162,"{7.242961669621194E-14,1.397341822947193E-5}"


,strain,date,date_bp,country,province,country_lat,country_lon,province_lat,province_lon,biovar,...,height_median,height_range,height,timetree_num_date,length_range,rate_95%_HPD,length_95%_HPD,rate_range,rate_median,length_median
sample,,,,,,,,,,,,,,,,,,,,,
GCA_008630455.1_ASM863045v1_genomic,C-826,2011.0,-10,Russia,Republic of Dagestan,64.6863,97.7453,43.0883,47.1499,Caucasica,...,3.0,"{2.999999999999659,3.0000000000002274}",3,2011,"{1.4273937684314273,44.852179285076105}","{2.6097465824417483E-9,6.321194324773121E-8}","{23.0251026957521,42.265688865474765}","{4.4171353372083285E-10,5.349289484410833E-7}",2.614544478805884E-8,29.66182276861643
GCA_008630415.1_ASM863041v1_genomic,C-672,1988.0,-33,Russia,Republic of Dagestan,64.6863,97.7453,43.0883,47.1499,Caucasica,...,26.0,"{25.99999999999966,26.000000000000227}",26,1988,"{0.0999317572162326,23.69872820107333}","{7.249413352729134E-8,1.6183423240693155E-6}","{0.3849389753981036,12.650602031963281}","{5.802709582733856E-8,1.694733787991764E-5}",3.4131437267050303E-7,6.86367764896692
GCA_006546745.1_ASM654674v1_genomic,I-3269,1990.0,-31,Mongolia,Bayan-Ölgii,46.825,103.85,48.547,89.8549,Ulegeica,...,24.0,"{23.999999999999773,24.000000000000227}",24,1990,"{3.366442708043376,76.18426571714122}","{7.048475896756691E-8,2.3683698306483924E-7}","{19.53422000238826,32.922449442003376}","{4.681916477954927E-8,1.6808983934032898E-6}",1.4883208765293748E-7,24.410008270255773
GCA_006546725.1_ASM654672v1_genomic,I-3196,1986.0,-35,Mongolia,Govi-Altai,46.825,103.85,45.3463,95.9326,Ulegeica,...,28.0,"{27.999999999999773,28.000000000000227}",28,1986,"{0.04291121951107968,86.74750643904615}","{2.0895228099157223E-8,1.4969767541491584E-7}","{14.286108804231745,34.74944354281658}","{8.823724288792062E-9,3.795225847417498E-5}",7.512125355179427E-8,21.449633713234775
GCA_006546665.1_ASM654666v1_genomic,I-3193,1987.0,-34,Mongolia,Khovd,46.825,103.85,48.0552,91.2673,Ulegeica,...,27.0,"{26.999999999999773,27.000000000000227}",27,1987,"{0.3664427080433761,48.908055263553806}","{6.793126768110123E-8,2.757304038486022E-7}","{13.10086710239635,26.103285610587037}","{3.599308325113401E-8,8.873055986291867E-6}",1.5246054297245548E-7,17.273348532222585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NODE79,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,30.29976917492388,"{14.955244317050486,47.9927667968382}",29.8877,1984.11,"{0.00156249379370621,35.41337498703514}","{4.164124831771047E-11,2.672094384408922E-7}","{0.00156249379370621,14.696016550014907}","{4.164124831771047E-11,9.014611514970941E-6}",2.3909672920519704E-8,4.639186880136492
NODE80,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,21.952472290115224,"{13.365015229852695,48.355588003336834}",22.4691,1991.53,"{0.036866763887143605,39.920587628336904}","{1.0488273462693892E-8,1.1994835376745965E-6}","{0.05819741258296318,17.349332562662205}","{1.0488273462693892E-8,2.8837272616003555E-5}",1.772764553018911E-7,6.4773424370369455
NODE81,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,27.70599160170255,"{14.64755034917772,44.49255052369415}",27.6095,1986.39,"{0.0011262655275672273,30.54458913823302}","{1.4255370435288464E-10,3.0569162872441313E-7}","{0.0011262655275672273,13.299656375518357}","{1.4255370435288464E-10,1.0531650076286934E-5}",3.013465807629524E-8,3.4804997915285334


,strain,date,date_bp,country,province,country_lat,country_lon,province_lat,province_lon,biovar,...,height_median,height_range,height,timetree_num_date,length_range,rate_95%_HPD,length_95%_HPD,rate_median,length_median,rate_range
sample,,,,,,,,,,,,,,,,,,,,,
SAMEA104233048,Post6,[-2007:-1882],[-4028:-3903],Germany,Bavaria,51.0834,10.4234,48.9468,11.4039,Bronze Age,...,NA,NA,258.500000,-1944.500000,"{0.26137597812328295,592.5612425268795}","{5.899275191756727E-9,6.628500059441837E-8}","{10.368349896758673,434.47299698367635}",1.1056113813575873E-8,279.7068901438097,"{1.7638360457688605E-9,5.162673494796717E-6}"
SAMEA104233049,1343UnTal85,[-2396:-2148],[-4417:-4169],Germany,Bavaria,51.0834,10.4234,48.9468,11.4039,Bronze Age,...,586.0,"{585.9999999999998,586.0000000000002}",586.000000,-2272.000000,"{0.019873221045259015,438.6664342603435}","{7.06178400605268E-9,4.901162112344934E-7}","{0.0365347912403422,286.9071437031581}",1.3738619000466807E-8,198.52007956847336,"{7.06178400605268E-9,2.6725273156174455E-4}"
SAMEA104233047,KunilaII,[-2574:-2340],[-4595:-4361],Estonia,Pärnu maakond,58.7524,25.3319,58.3195,24.3026,Bronze Age,...,771.0,"{770.9999999999998,771.0000000000002}",771.000000,-2457.000000,"{15.516357303293034,579.4696931574999}","{5.914885313936088E-9,1.664021441329337E-8}","{212.14118263263276,322.9907129516921}",1.0634238460469731E-8,247.73285748072288,"{3.553982434339283E-11,3.1476740218664647E-7}"
SAMEA104233046,Gyvakarai1,[-2621:-2472],[-4642:-4493],Lithuania,Panevezys County,55.35,23.75,55.9156,25.0312,Bronze Age,...,860.5,"{860.4999999999998,860.5000000000002}",860.500000,-2546.500000,"{124.26191842019148,794.1352557281409}","{5.954375754189968E-9,1.8556102757533683E-8}","{137.0963765275194,333.62429982253525}",1.084629076731244E-8,234.42400727706092,"{1.681707147922761E-11,5.143391916110305E-8}"
SAMEA3541827,RISE509,[-2876:-2677],[-4897:-4698],Russia,Krasnoyarsk Krai,64.6863,97.7453,63.3234,97.098,Bronze Age,...,1090.5,"{1090.4999999999998,1090.5000000000002}",1090.500000,-2776.500000,"{0.009520840034610956,631.4029061404672}","{9.43993461581082E-12,3.855538614541085E-6}","{0.009520840034610956,195.1582901865247}",2.7835411003429926E-7,35.8665897510715,"{9.43993461581082E-12,2.4017124916796598E-4}"
SAMEA3541826,RISE505,[-1746:-1626],[-3767:-3647],Russia,Altai Krai,64.6863,97.7453,52.6932,82.6931,Bronze Age,...,NA,NA,0.000000,-1686.000000,"{258.7613759781233,851.0612425268795}","{4.175019264222575E-8,1.2997492607400134E-7}","{268.8683498967587,692.9729969836764}",6.594314426461917E-8,538.2068901438097,"{3.7354052936568944E-8,1.7900722210928818E-7}"
SAMEA6812877,GLZ002,[-2569:-2467],[-4590:-4488],Russia,Irkutsk Oblast,64.6863,97.7453,56.637,104.719,Bronze Age,...,832.0,"{831.9999999999998,832.0000000000002}",832.000000,-2518.000000,"{148.50405789372724,216.9712877158113}","{6.458078866578565E-12,1.4976644269696073E-8}","{148.50405789372724,168.64795492796281}",9.858374234245279E-9,152.3608818717251,"{6.458078866578565E-12,5.143391916110305E-8}"
SAMEA6812876,GLZ001,[-2838:-2495],[-4859:-4516],Russia,Irkutsk Oblast,64.6863,97.7453,56.637,104.719,Bronze Age,...,980.5,"{980.4999999999998,980.5000000000002}",980.500000,-2666.500000,"{0.004057893727235751,68.47128771581129}","{6.634839520346167E-10,1.8886659867901838E-6}","{0.004057893727235751,20.147954927962815}",1.4360584565765109E-7,3.8608818717250983,"{6.634839520346167E-10,1.9629055065027202E-4}"
NODE0,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,1143.3570853433978,"{1090.5095208400346,1721.9029061404672}",1166.091245,-2852.091245,NA,NA,NA,NA,NA,NA


## Reduced Dataframe for Auspice

In [11]:
# OPtions
print(tree_dict[branch]["df"].columns)

Index(['strain', 'date', 'date_bp', 'country', 'province', 'country_lat',
       'country_lon', 'province_lat', 'province_lon', 'biovar', 'branch_major',
       'branch_minor', 'biosample_accession', 'biosample_comment',
       'branch_number', 'continent', 'date_mean', 'date_bp_mean', 'date_err',
       'lat', 'lon', 'host_human', 'node_type', 'branch_length', 'rate',
       'rate_subs', 'timetree_num_date_confidence', 'height_95%_HPD', 'length',
       'posterior', 'height_median', 'height_range', 'height',
       'timetree_num_date', 'length_range', 'rate_95%_HPD', 'length_95%_HPD',
       'rate_median', 'length_median', 'rate_range'],
      dtype='object')


In [12]:
for branch in tree_dict:
    
    columns = [
        # Node type is mandatory
        "node_type",        
        # Draw Divergence Tree
        "branch_length",
        # Draw Time Tree
        "timetree_num_date",
        "timetree_num_date_confidence",    
        "rate",
        "rate_subs",
        # Geo
        "country",
        "province",
        "country_lat",
        "country_lon",
        "province_lat",
        "province_lon",        
        # Text descriptions
        "strain",
        "branch_major",
        "biovar",
        "host_human",
        "date",
        "date_bp",        
    ]
    
    auspice_df = copy.copy(tree_dict[branch]["df"][columns])
    auspice_df.rename(columns={"rate": "rate_beast"}, inplace=True)
    auspice_df.rename(columns={"rate_subs": "rate_subs_beast"}, inplace=True)
    tree_dict[branch]["auspice_df"] = auspice_df
    display(tree_dict[branch]["auspice_df"])

,node_type,branch_length,timetree_num_date,timetree_num_date_confidence,rate_beast,rate_subs_beast,country,province,country_lat,country_lon,province_lat,province_lon,strain,branch_major,biovar,host_human,date,date_bp
sample,,,,,,,,,,,,,,,,,,
Reference,terminal,17.516688,1992.000000,"[1992.0, 1992.0]",3.553717e-07,1.502902,United States of America,Colorado,39.7837,-100.446,38.7252,-105.608,CO92,1.ORI,Orientalis,Human,1992,-29
GCA_000834775.1_ASM83477v1_genomic,terminal,9.482611,1967.000000,"[1967.0, 1967.0]",3.123540e-07,1.320976,United States of America,Arizona,39.7837,-100.446,34.3953,-111.763,Dodson,1.ORI,Orientalis,Human,1967.0,-54
GCA_000834335.1_ASM83433v1_genomic,terminal,4.917912,1954.000000,"[1954.0, 1954.0]",2.122180e-07,0.897491,United States of America,California,39.7837,-100.446,36.7015,-118.756,Shasta,1.ORI,Orientalis,Human,1954.0,-67
GCA_000169635.1_ASM16963v1_genomic,terminal,32.332497,2005.000000,"[2005.0, 2005.0]",1.026717e-07,0.434209,Madagascar,NA,-18.925,46.4416,NA,NA,MG05-1020,1.ORI,Orientalis,Human,2005.0,-16
GCA_000170275.1_ASM17027v1_genomic,terminal,18.461812,1991.000000,"[1991.0, 1991.0]",8.033665e-08,0.339752,China,Yunnan,35.0001,105,25,102,F1991016,1.ORI,Orientalis,Non-Human,1991.0,-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NODE111,internal,0.194478,2004.241372,"[2007.5041318052265, 2000.475519086657]",1.016505e-07,0.429890,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NODE112,internal,0.929602,2006.999750,"[2009.6153588063687, 2003.8227971142646]",1.311277e-07,0.554552,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NODE113,internal,0.351704,2005.846571,"[2008.677776298874, 2002.70281589384]",1.653129e-07,0.699124,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


,node_type,branch_length,timetree_num_date,timetree_num_date_confidence,rate_beast,rate_subs_beast,country,province,country_lat,country_lon,province_lat,province_lon,strain,branch_major,biovar,host_human,date,date_bp
sample,,,,,,,,,,,,,,,,,,
GCA_000324785.1_ASM32478v1_genomic,terminal,7.304944,1979.000000,"[1979.0, 1979.0]",2.189105e-07,0.925794,China,Yunnan,35.0001,105,25,102,E1979001,1.IN,Antiqua,Non-Human,1979.0,-42
GCA_000022845.1_ASM2284v1_genomic,terminal,3.860503,1976.000000,"[1976.0, 1976.0]",3.444121e-07,1.456553,China,Tibet,35.0001,105,29.8556,90.875,Z176003,1.IN,Antiqua,Non-Human,1976.0,-45
GCA_000022825.1_ASM2282v1_genomic,terminal,31.757595,1982.000000,"[1982.0, 1982.0]",2.079253e-06,8.793364,China,Yunnan,35.0001,105,25,102,D182038,1.IN,Antiqua,Non-Human,1982.0,-39
GCA_000022805.1_ASM2280v1_genomic,terminal,70.350981,2006.000000,"[2006.0, 2006.0]",1.477245e-07,0.624742,China,Yunnan,35.0001,105,25,102,D106004,1.IN,Antiqua,Non-Human,2006.0,-15
GCA_000325465.1_ASM32546v1_genomic,terminal,9.419908,1957.000000,"[1957.0, 1957.0]",2.958604e-07,1.251223,China,Yunnan,35.0001,105,25,102,YN472,1.IN,Antiqua,Non-Human,1957.0,-64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NODE33,internal,8.520717,1972.797250,"[1986.4898330318686, 1957.7256089593875]",5.422988e-08,0.229343,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NODE34,internal,3.206524,1981.521412,"[1988.6716159399791, 1973.0440875505783]",2.454978e-07,1.038234,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NODE35,internal,9.000596,1987.215157,"[1993.659574537487, 1980.308465360717]",1.360054e-07,0.575180,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


,node_type,branch_length,timetree_num_date,timetree_num_date_confidence,rate_beast,rate_subs_beast,country,province,country_lat,country_lon,province_lat,province_lon,strain,branch_major,biovar,host_human,date,date_bp
sample,,,,,,,,,,,,,,,,,,
GCA_000169615.1_ASM16961v1_genomic,terminal,51.265788,2004.000000,"[0, 0]",6.444323e-08,0.272537,Uganda,NA,1.53336,32.2167,NA,NA,UG05-0454,1.ANT,Antiqua,Human,2004.0,-17
GCA_003798225.1_ASM379822v1_genomic,terminal,2.045508,1954.000000,"[1954.0, 1954.0]",4.756261e-07,2.011469,Democratic Republic of the Congo,NA,-2.98143,23.8223,NA,NA,FDAARGOS_601,1.ANT,Antiqua,NA,1954.0,-67
GCA_000835005.1_ASM83500v1_genomic,terminal,236.991771,1985.000000,"[1985.0, 1985.0]",6.444323e-08,0.272537,Kenya,Nairobi,1.44197,38.4314,-1.30317,36.8261,Nairobi,1.ANT,Antiqua,NA,1985.0,-36
GCA_000013825.1_ASM1382v1_genomic,terminal,12.265788,1965.000000,"[1965.0, 1965.0]",1.159070e-07,0.490182,Democratic Republic of the Congo,NA,-2.98143,23.8223,NA,NA,Antiqua,1.ANT,Antiqua,Human,1965.0,-56
NODE0,internal,0.000000,1758.910228,"[1855.0439441412525, 1644.8345812291716]",0.000000e+00,0.000000,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NODE1,internal,203.946262,1950.327725,"[1953.999874244536, 1943.1927174315974]",6.444323e-08,0.272537,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NODE2,internal,0.779720,1954.969051,"[1964.7555744433698, 1948.8644061878467]",2.517850e-07,1.064824,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


,node_type,branch_length,timetree_num_date,timetree_num_date_confidence,rate_beast,rate_subs_beast,country,province,country_lat,country_lon,province_lat,province_lon,strain,branch_major,biovar,host_human,date,date_bp
sample,,,,,,,,,,,,,,,,,,
SAMEA5818830,terminal,0.080411,1560.000000,"[1560.0, 1560.0]",3.588157e-08,0.151747,Switzerland,Nidwalden,46.8133,8.44495,46.9428,8.41198,STN021,1.PRE,Second Pandemic,Human,[1485:1635],[-536:-386]
SAMEA5818829,terminal,1.728035,1560.000000,"[1560.0, 1560.0]",3.372235e-08,0.142615,Switzerland,Nidwalden,46.8133,8.44495,46.9428,8.41198,STN020,1.PRE,Second Pandemic,Human,[1485:1635],[-536:-386]
SAMEA5818828,terminal,1.728035,1560.000000,"[1560.0, 1560.0]",3.454374e-08,0.146089,Switzerland,Nidwalden,46.8133,8.44495,46.9428,8.41198,STN019,1.PRE,Second Pandemic,Human,[1485:1635],[-536:-386]
SAMEA5818826,terminal,0.023473,1560.000000,"[1560.0, 1560.0]",3.387151e-08,0.143246,Switzerland,Nidwalden,46.8133,8.44495,46.9428,8.41198,STN014,1.PRE,Second Pandemic,Human,[1485:1635],[-536:-386]
SAMEA5818825,terminal,0.080411,1560.000000,"[1560.0, 1560.0]",1.428955e-07,0.604319,Switzerland,Nidwalden,46.8133,8.44495,46.9428,8.41198,STN013,1.PRE,Second Pandemic,Human,[1485:1635],[-536:-386]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NODE34,internal,2.944394,1631.712419,"[1647.1824752526363, 1614.2625699793778]",5.042076e-08,0.213234,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NODE35,internal,80.544459,1690.388727,"[1716.0895381907403, 1655.2814046941703]",1.671410e-08,0.070686,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NODE36,internal,17.041200,1708.538003,"[1719.1929733173438, 1693.4645716880063]",5.247133e-08,0.221906,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


,node_type,branch_length,timetree_num_date,timetree_num_date_confidence,rate_beast,rate_subs_beast,country,province,country_lat,country_lon,province_lat,province_lon,strain,branch_major,biovar,host_human,date,date_bp
sample,,,,,,,,,,,,,,,,,,
GCA_009909635.1_ASM990963v1_genomic,terminal,13.009036,1923.000000,"[1923.0, 1923.0]",1.678153e-07,0.709707,Russia,Rostov Oblast,64.6863,97.7453,47.6222,40.7958,9_10,2.MED,Medievalis,Human,1923.0,-98
GCA_009296005.1_ASM929600v1_genomic,terminal,0.469889,1953.000000,"[1953.0, 1953.0]",8.264115e-08,0.349498,Russia,Chechnya,64.6863,97.7453,43.3976,45.6985,C-25,2.MED,Medievalis,Non-Human,1953.0,-68
GCA_008630485.1_ASM863048v1_genomic,terminal,3.250880,1997.000000,"[1997.0, 1996.9999999999998]",7.262572e-07,3.071413,Russia,Kabardino-Balkaria,64.6863,97.7453,43.4428,43.4205,C-742,2.MED,Medievalis,Non-Human,1997.0,-24
GCA_008630435.1_ASM863043v1_genomic,terminal,26.020483,1996.000000,"[1996.0, 1996.0]",6.860444e-07,2.901349,Russia,Karachay-Cherkessia,64.6863,97.7453,43.7368,41.7268,C-719,2.MED,Medievalis,Non-Human,1996.0,-25
GCA_008630395.1_ASM863039v1_genomic,terminal,4.468999,1984.000000,"[1984.0, 1983.9999999999998]",5.818262e-07,2.460600,Russia,Republic of Dagestan,64.6863,97.7453,43.0883,47.1499,C-528,2.MED,Medievalis,Non-Human,1984.0,-37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NODE110,internal,6.738293,1988.877114,"[1995.1945509657712, 1981.978551608946]",1.440738e-07,0.609302,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NODE111,internal,9.988554,2004.492834,"[2008.9988057840776, 1995.7386247694087]",7.567338e-08,0.320030,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NODE112,internal,2.784600,1991.215981,"[1996.0199169572434, 1985.5990761703022]",5.944240e-08,0.251388,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


...

,node_type,branch_length,timetree_num_date,timetree_num_date_confidence,rate_beast,rate_subs_beast,country,province,country_lat,country_lon,province_lat,province_lon,strain,branch_major,biovar,host_human,date,date_bp
sample,,,,,,,,,,,,,,,,,,
GCA_000325305.1_ASM32530v1_genomic,terminal,4.884258,1998.000000,"[1998.0, 1998.0]",5.196203e-08,0.219753,Mongolia,Govi-Altai,46.825,103.85,45.3463,95.9326,MGJZ9,3.ANT,Antiqua,Non-Human,1998.0,-23
GCA_000325285.1_ASM32528v1_genomic,terminal,4.504057,1997.000000,"[1997.0, 1997.0]",4.693787e-07,1.985049,Mongolia,Dornogovi,46.825,103.85,44.418,109.778,MGJZ7,3.ANT,Antiqua,Non-Human,1997.0,-24
GCA_000325265.1_ASM32526v1_genomic,terminal,4.504057,1997.000000,"[1997.0, 1997.0]",1.792326e-07,0.757992,Mongolia,Dornogovi,46.825,103.85,44.418,109.778,MGJZ6,3.ANT,Antiqua,Non-Human,1997.0,-24
GCA_000325245.1_ASM32524v1_genomic,terminal,3.242909,1980.000000,"[1980.0, 1980.0]",4.604132e-06,19.471327,Mongolia,Govi-Altai,46.825,103.85,45.3463,95.9326,MGJZ3,3.ANT,Antiqua,Non-Human,1980.0,-41
GCA_000325205.1_ASM32520v1_genomic,terminal,6.884258,2000.000000,"[2000.0, 2000.0]",1.895825e-07,0.801763,Mongolia,Bayan-Ölgii,46.825,103.85,48.547,89.8549,MGJZ11,3.ANT,Antiqua,Non-Human,2000.0,-21
GCA_000324325.1_ASM32432v1_genomic,terminal,12.036414,1961.000000,"[1961.0, 1961.0]",3.970867e-07,1.679319,China,Gansu,35.0001,105,38,102,CMCC71001,3.ANT,Antiqua,Non-Human,1961.0,-60
GCA_000323765.1_ASM32376v1_genomic,terminal,9.721393,1976.000000,"[1976.0, 1976.0]",3.723379e-07,1.574654,China,Gansu,35.0001,105,38,102,C1976001,3.ANT,Antiqua,Non-Human,1976.0,-45
GCA_000323525.1_ASM32352v1_genomic,terminal,22.721393,1989.000000,"[1989.0, 1989.0]",4.556191e-08,0.192686,China,Gansu,35.0001,105,38,102,71021,3.ANT,Antiqua,Non-Human,1989.0,-32
GCA_000323505.1_ASM32350v1_genomic,terminal,47.714694,2004.000000,"[2004.0, 2004.0]",4.412225e-08,0.186597,China,Qinghai,35.0001,105,35.4071,95.9521,7,3.ANT,Antiqua,Human,2004.0,-17


,node_type,branch_length,timetree_num_date,timetree_num_date_confidence,rate_beast,rate_subs_beast,country,province,country_lat,country_lon,province_lat,province_lon,strain,branch_major,biovar,host_human,date,date_bp
sample,,,,,,,,,,,,,,,,,,
GCA_009669545.1_ASM966954v1_genomic,terminal,5.410463,2006.000000,"[2006.0000000000002, 2006.0]",1.570504e-08,0.066418,China,Xinjiang,35.0001,105,42.4805,85.4633,42126,0.ANT,Antiqua,Non-Human,2006.0,-15
GCA_009669555.1_ASM966955v1_genomic,terminal,10.677114,2005.000000,"[2005.0000000000002, 2004.9999999999998]",3.697239e-08,0.156360,China,Xinjiang,35.0001,105,42.4805,85.4633,42123,0.ANT,Antiqua,Non-Human,2005.0,-16
GCA_009669565.1_ASM966956v1_genomic,terminal,2.117711,2005.000000,"[2005.0000000000002, 2005.0]",5.413330e-08,0.228935,China,Xinjiang,35.0001,105,42.4805,85.4633,42118,0.ANT,Antiqua,Non-Human,2005.0,-16
GCA_009669605.1_ASM966960v1_genomic,terminal,2.117711,2005.000000,"[2005.0, 2004.9999999999998]",3.525254e-08,0.149086,China,Xinjiang,35.0001,105,42.4805,85.4633,42117,0.ANT,Antiqua,Non-Human,2005.0,-16
GCA_009669625.1_ASM966962v1_genomic,terminal,13.851436,2005.000000,"[2005.0, 2005.0]",1.210787e-08,0.051205,China,Xinjiang,35.0001,105,42.4805,85.4633,42116,0.ANT,Antiqua,Non-Human,2005.0,-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NODE97,internal,2.563553,1995.240889,"[1998.99577465624, 1987.4546828191246]",2.856819e-08,0.120818,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NODE98,internal,2.375887,1998.403159,"[2000.997431030353, 1991.859927842856]",2.572350e-08,0.108787,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NODE99,internal,1.208979,1993.725629,"[2000.136072806417, 1984.0716057453021]",1.762941e-08,0.074557,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


,node_type,branch_length,timetree_num_date,timetree_num_date_confidence,rate_beast,rate_subs_beast,country,province,country_lat,country_lon,province_lat,province_lon,strain,branch_major,biovar,host_human,date,date_bp
sample,,,,,,,,,,,,,,,,,,
SAMEA5661390,terminal,2.897253,478.000000,"[478.0000000000001, 477.9999999999999]",1.459943e-06,6.174240,Germany,Bavaria,51.0834,10.4234,48.9468,11.4039,UNT004,0.ANT,Antiqua,Human,[419:537],[-1602:-1484]
SAMEA5661389,terminal,16.676053,516.000000,"[516.0000000000001, 515.9999999999999]",3.461309e-09,0.014638,Germany,Bavaria,51.0834,10.4234,48.9468,11.4039,UNT003,0.ANT,Antiqua,Human,[433:599],[-1588:-1422]
SAMEA5661385,terminal,24.099582,517.000000,"[517.0000000000001, 516.9999999999999]",7.160345e-07,3.028180,Germany,Bavaria,51.0834,10.4234,48.9468,11.4039,PET004,0.ANT,Antiqua,Human,[434:600],[-1587:-1421]
SAMEA5661384,terminal,96.812017,521.000000,"[521.0000000000001, 520.9999999999999]",1.858864e-08,0.078613,Spain,Valencia Community,39.3261,-4.83798,39.682,-0.765441,VAL001,0.ANT,Antiqua,Human,[432:610],[-1589:-1411]
SAMEA5661372,terminal,6.632183,765.000000,"[0, 0]",9.442931e-08,0.399351,France,Centre-Loire Valley,46.6034,1.88833,47.549,1.73241,LSD020,0.ANT,Antiqua,Human,[650:880],[-1371:-1141]
SAMEA5661367,terminal,6.632183,765.000000,"[0, 0]",1.283400e-07,0.542762,France,Centre-Loire Valley,46.6034,1.88833,47.549,1.73241,LSD001,0.ANT,Antiqua,Human,[650:880],[-1371:-1141]
SAMEA5661365,terminal,13.841901,575.000000,"[575.0000000000001, 574.9999999999999]",3.400581e-08,0.143814,England,East of England,52.531,-1.26491,52.22,0.487578,EDI004,0.ANT,Antiqua,Human,[500:650],[-1521:-1371]
SAMEA5661364,terminal,13.841901,575.000000,"[575.0000000000001, 574.9999999999999]",1.977523e-07,0.836314,England,East of England,52.531,-1.26491,52.22,0.487578,EDI003,0.ANT,Antiqua,Human,[500:650],[-1521:-1371]
SAMEA5661363,terminal,2.289918,555.500000,"[555.5000000000001, 555.4999999999999]",2.059089e-08,0.087081,England,East of England,52.531,-1.26491,52.22,0.487578,EDI001,0.ANT,Antiqua,Human,[474:637],[-1547:-1384]


,node_type,branch_length,timetree_num_date,timetree_num_date_confidence,rate_beast,rate_subs_beast,country,province,country_lat,country_lon,province_lat,province_lon,strain,branch_major,biovar,host_human,date,date_bp
sample,,,,,,,,,,,,,,,,,,
GCA_008630455.1_ASM863045v1_genomic,terminal,29.2825,2011,"[2011.0, 2011.0]",3.0165e-08,0.127571,Russia,Republic of Dagestan,64.6863,97.7453,43.0883,47.1499,C-826,0.PE,Caucasica,Non-Human,2011.0,-10
GCA_008630415.1_ASM863041v1_genomic,terminal,6.28249,1988,"[1988.0, 1988.0]",5.68933e-07,2.40607,Russia,Republic of Dagestan,64.6863,97.7453,43.0883,47.1499,C-672,0.PE,Caucasica,Non-Human,1988.0,-33
GCA_006546745.1_ASM654674v1_genomic,terminal,23.105,1990,"[1990.0, 1990.0]",1.54868e-07,0.654951,Mongolia,Bayan-Ölgii,46.825,103.85,48.547,89.8549,I-3269,0.PE,Ulegeica,Non-Human,1990.0,-31
GCA_006546725.1_ASM654672v1_genomic,terminal,23.8653,1986,"[1986.0, 1986.0]",1.12831e-07,0.477172,Mongolia,Govi-Altai,46.825,103.85,45.3463,95.9326,I-3196,0.PE,Ulegeica,Non-Human,1986.0,-35
GCA_006546665.1_ASM654666v1_genomic,terminal,20.4987,1987,"[1987.0, 1987.0]",1.64816e-07,0.697021,Mongolia,Khovd,46.825,103.85,48.0552,91.2673,I-3193,0.PE,Ulegeica,Non-Human,1987.0,-34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NODE79,internal,9.1079,1984.11,"[1992.827346687587, 1977.0886858553683]",7.64583e-08,0.32335,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NODE80,internal,4.37041,1991.53,"[1999.4364370064268, 1981.778606751268]",3.8192e-07,1.61518,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NODE81,internal,3.51422,1986.39,"[1994.975657130774, 1978.653294424603]",9.264e-08,0.391784,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


,node_type,branch_length,timetree_num_date,timetree_num_date_confidence,rate_beast,rate_subs_beast,country,province,country_lat,country_lon,province_lat,province_lon,strain,branch_major,biovar,host_human,date,date_bp
sample,,,,,,,,,,,,,,,,,,
SAMEA104233048,terminal,21.790965,-1944.500000,"[0, 0]",2.660027e-08,0.112495,Germany,Bavaria,51.0834,10.4234,48.9468,11.4039,Post6,0.PRE,Bronze Age,Human,[-2007:-1882],[-4028:-3903]
SAMEA104233049,terminal,50.342757,-2272.000000,"[-2272.0, -2272.0]",2.153232e-07,0.910623,Germany,Bavaria,51.0834,10.4234,48.9468,11.4039,1343UnTal85,0.PRE,Bronze Age,Human,[-2396:-2148],[-4417:-4169]
SAMEA104233047,terminal,236.800707,-2457.000000,"[-2457.0, -2457.0]",1.069577e-08,0.045233,Estonia,Pärnu maakond,58.7524,25.3319,58.3195,24.3026,KunilaII,0.PRE,Bronze Age,Human,[-2574:-2340],[-4595:-4361]
SAMEA104233046,terminal,221.689649,-2546.500000,"[-2546.5, -2546.5]",1.120606e-08,0.047392,Lithuania,Panevezys County,55.35,23.75,55.9156,25.0312,Gyvakarai1,0.PRE,Bronze Age,Human,[-2621:-2472],[-4642:-4493]
SAMEA3541827,terminal,9.325354,-2776.500000,"[-2776.5, -2776.5]",1.131250e-06,4.784166,Russia,Krasnoyarsk Krai,64.6863,97.7453,63.3234,97.098,RISE509,0.PRE,Bronze Age,Human,[-2876:-2677],[-4897:-4698]
SAMEA3541826,terminal,280.290965,-1686.000000,"[0, 0]",7.449197e-08,0.315034,Russia,Altai Krai,64.6863,97.7453,52.6932,82.6931,RISE505,0.PRE,Bronze Age,Human,[-1746:-1626],[-3767:-3647]
SAMEA6812877,terminal,150.119559,-2518.000000,"[-2518.0, -2518.0]",8.381519e-09,0.035446,Russia,Irkutsk Oblast,64.6863,97.7453,56.637,104.719,GLZ002,0.PRE,Bronze Age,Human,[-2569:-2467],[-4590:-4488]
SAMEA6812876,terminal,1.619559,-2666.500000,"[-2666.5, -2666.5]",5.942839e-07,2.513285,Russia,Irkutsk Oblast,64.6863,97.7453,56.637,104.719,GLZ001,0.PRE,Bronze Age,Human,[-2838:-2495],[-4859:-4516]
NODE0,internal,0.000000,-2852.091245,"[-2776.5095208400344, -3000.1689872318607]",0.000000e+00,0.000000,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


---

# Plot Rates

In [13]:
"""rates = [float(rate) for rate in list(tree_dict["1.ORI"]["df"]["rate"]) if rate != NO_DATA_CHAR]
print(sum(rates) / len(rates))
sns.histplot(rates)"""


'rates = [float(rate) for rate in list(tree_dict["1.ORI"]["df"]["rate"]) if rate != NO_DATA_CHAR]\nprint(sum(rates) / len(rates))\nsns.histplot(rates)'

---

# Export

## Time Tree

In [14]:
for branch in tree_dict:
    out_timetree = copy.deepcopy(tree_dict[branch]["tree"])

    metadata_to_comment(out_timetree, tree_dict[branch]["df"])    
    out_timetree_nex_path = os.path.join(tree_dir, branch + ".timetree.nex")
    Phylo.write(out_timetree, out_timetree_nex_path, "nexus")

    for c in out_timetree.find_clades():
        c.comment = None

    out_timetree_nwk_path = os.path.join(tree_dir, branch + ".timetree.nwk")
    Phylo.write(out_timetree, out_timetree_nwk_path, "newick")

## Augur

In [15]:
for branch in tree_dict:
    
    augur_dict = augur_export(
        tree_path=None, 
        aln_path=None,  
        tree=tree_dict[branch]["tree"], 
        tree_df=tree_dict[branch]["auspice_df"], 
        color_keyword_exclude=["geometry"],
        type_convert = {
            "branch_number" : (lambda x : str(x))
        },
    )
    
    tree_dict[branch]["augur_dict"] = augur_dict
    
    first_taxa = list(augur_dict["nodes"].keys())[0]
    print(augur_dict["nodes"][first_taxa])

    out_path_augur_json = os.path.join(tree_dir, branch + "_augur.json" )
    utils.write_json(data=tree_dict[branch]["augur_dict"], file_name=out_path_augur_json, indent=JSON_INDENT)
    tree_dict[branch]["augur_json_path"] = out_path_augur_json

{'node_type': 'internal', 'branch_length': 0.0, 'num_date': 1867.1884218560415, 'num_date_confidence': [1910.50330129055, 1801.597968774552], 'rate_beast': 0.0, 'rate_subs_beast': 0.0, 'country': 'NA', 'province': 'NA', 'country_lat': 'NA', 'country_lon': 'NA', 'province_lat': 'NA', 'province_lon': 'NA', 'strain': 'NA', 'branch_major': 'NA', 'biovar': 'NA', 'host_human': 'NA', 'date': 'NA', 'date_bp': 'NA'}
{'node_type': 'internal', 'branch_length': 0.0, 'num_date': 1824.653130788105, 'num_date_confidence': [1931.4884503335472, 1648.352772485698], 'rate_beast': 0.0, 'rate_subs_beast': 0.0, 'country': 'NA', 'province': 'NA', 'country_lat': 'NA', 'country_lon': 'NA', 'province_lat': 'NA', 'province_lon': 'NA', 'strain': 'NA', 'branch_major': 'NA', 'biovar': 'NA', 'host_human': 'NA', 'date': 'NA', 'date_bp': 'NA'}
{'node_type': 'internal', 'branch_length': 0.0, 'num_date': 1758.9102280437203, 'num_date_confidence': [1855.0439441412525, 1644.8345812291716], 'rate_beast': 0.0, 'rate_subs_be

## Auspice

In [23]:
for branch in tree_dict:
    
    # Store the color
    if branch == "0.ANT4":
        branch_major_color = colors_dict["branch_major"]["0.ANT"]
    else:
        branch_major_color = colors_dict["branch_major"][branch]

    auspice_dict = auspice_export(
        tree=tree_dict[branch]["tree"],
        augur_json_paths=tree_dict[branch]["augur_json_path"], 
        auspice_config_path=auspice_config_path, 
        auspice_colors_path=out_path_colors,
        auspice_latlons_path=out_path_latlon, 
        )


    label_col = list(tree_dict[branch]["auspice_df"])
    print(label_col)

    # Recursively add branch attrs
    branch_attributes(
        tree_dict=auspice_dict["tree"], 
        sub_dict=auspice_dict["tree"], 
        df=tree_dict[branch]["auspice_df"],
        label_col=label_col,
        )
    
    
    # Last manual changes
    auspice_dict_copy = copy.deepcopy(auspice_dict)
    for i in range(0, len(auspice_dict_copy["meta"]["colorings"])):
        coloring = auspice_dict_copy["meta"]["colorings"][i]
        for key in coloring:
            # Node type as internal or terminal
            if coloring[key] == "node_type":
                auspice_dict["meta"]["colorings"][i]['scale'] = [['internal', '#FFFFFF'], ['terminal', branch_major_color]]
                #print(auspice_dict["meta"]["colorings"][i])
            # Confidence category
            if "conf_category" in coloring[key]:
                auspice_dict["meta"]["colorings"][i]['scale'] = [['LOW', '#FFFFFF'], ['HIGH', branch_major_color]]
                #print(auspice_dict["meta"]["colorings"][i])

    # Write outputs - For Local Rendering
    out_path_auspice_local_json = os.path.join(tree_dir, branch + ".json" )
    utils.write_json(data=auspice_dict, file_name=out_path_auspice_local_json, indent=JSON_INDENT, include_version=False)
    export_v2.validate_data_json(out_path_auspice_local_json)
    print("Validation successful for local JSON.\n")

Validating schema of '/mnt/c/Users/ktmea/Projects/plague-phylogeography-projects/main/config/auspice_config.json'...
Validation success.
['node_type', 'branch_length', 'timetree_num_date', 'timetree_num_date_confidence', 'rate_beast', 'rate_subs_beast', 'country', 'province', 'country_lat', 'country_lon', 'province_lat', 'province_lon', 'strain', 'branch_major', 'biovar', 'host_human', 'date', 'date_bp']


Validating produced JSON
Validating schema of '/mnt/c/Users/ktmea/Projects/plague-phylogeography-projects/main/beast/all/chromosome/clade/summary_trees_noHyperPrior/relaxed_clock/1.ORI.json'...
Validating that the JSON is internally consistent...
Validation of '/mnt/c/Users/ktmea/Projects/plague-phylogeography-projects/main/beast/all/chromosome/clade/summary_trees_noHyperPrior/relaxed_clock/1.ORI.json' succeeded, but there were warnings you may want to resolve.
Validation successful for local JSON.

Validating schema of '/mnt/c/Users/ktmea/Projects/plague-phylogeography-projects/main/config/auspice_config.json'...
Validation success.
['node_type', 'branch_length', 'timetree_num_date', 'timetree_num_date_confidence', 'rate_beast', 'rate_subs_beast', 'country', 'province', 'country_lat', 'country_lon', 'province_lat', 'province_lon', 'strain', 'branch_major', 'biovar', 'host_human', 'date', 'date_bp']
Validating produced JSON
Validating schema of '/mnt/c/Users/ktmea/Projects/plague-phylo

Validating that the JSON is internally consistent...
Validation of '/mnt/c/Users/ktmea/Projects/plague-phylogeography-projects/main/beast/all/chromosome/clade/summary_trees_noHyperPrior/relaxed_clock/1.IN.json' succeeded, but there were warnings you may want to resolve.
Validation successful for local JSON.

Validating schema of '/mnt/c/Users/ktmea/Projects/plague-phylogeography-projects/main/config/auspice_config.json'...
Validation success.
['node_type', 'branch_length', 'timetree_num_date', 'timetree_num_date_confidence', 'rate_beast', 'rate_subs_beast', 'country', 'province', 'country_lat', 'country_lon', 'province_lat', 'province_lon', 'strain', 'branch_major', 'biovar', 'host_human', 'date', 'date_bp']
Validating produced JSON
Validating schema of '/mnt/c/Users/ktmea/Projects/plague-phylogeography-projects/main/beast/all/chromosome/clade/summary_trees_noHyperPrior/relaxed_clock/1.ANT.json'...
Validating that the JSON is internally consistent...
Validation of '/mnt/c/Users/ktmea/P

Validating produced JSON
Validating schema of '/mnt/c/Users/ktmea/Projects/plague-phylogeography-projects/main/beast/all/chromosome/clade/summary_trees_noHyperPrior/relaxed_clock/1.PRE.json'...
Validating that the JSON is internally consistent...
Validation of '/mnt/c/Users/ktmea/Projects/plague-phylogeography-projects/main/beast/all/chromosome/clade/summary_trees_noHyperPrior/relaxed_clock/1.PRE.json' succeeded, but there were warnings you may want to resolve.
Validation successful for local JSON.

Validating schema of '/mnt/c/Users/ktmea/Projects/plague-phylogeography-projects/main/config/auspice_config.json'...
Validation success.
['node_type', 'branch_length', 'timetree_num_date', 'timetree_num_date_confidence', 'rate_beast', 'rate_subs_beast', 'country', 'province', 'country_lat', 'country_lon', 'province_lat', 'province_lon', 'strain', 'branch_major', 'biovar', 'host_human', 'date', 'date_bp']


Validating produced JSON
Validating schema of '/mnt/c/Users/ktmea/Projects/plague-phylogeography-projects/main/beast/all/chromosome/clade/summary_trees_noHyperPrior/relaxed_clock/2.MED.json'...
Validating that the JSON is internally consistent...
Validation of '/mnt/c/Users/ktmea/Projects/plague-phylogeography-projects/main/beast/all/chromosome/clade/summary_trees_noHyperPrior/relaxed_clock/2.MED.json' succeeded, but there were warnings you may want to resolve.
Validation successful for local JSON.

Validating schema of '/mnt/c/Users/ktmea/Projects/plague-phylogeography-projects/main/config/auspice_config.json'...
Validation success.


['node_type', 'branch_length', 'timetree_num_date', 'timetree_num_date_confidence', 'rate_beast', 'rate_subs_beast', 'country', 'province', 'country_lat', 'country_lon', 'province_lat', 'province_lon', 'strain', 'branch_major', 'biovar', 'host_human', 'date', 'date_bp']
Validating produced JSON
Validating schema of '/mnt/c/Users/ktmea/Projects/plague-phylogeography-projects/main/beast/all/chromosome/clade/summary_trees_noHyperPrior/relaxed_clock/2.ANT.json'...
Validating that the JSON is internally consistent...
Validation of '/mnt/c/Users/ktmea/Projects/plague-phylogeography-projects/main/beast/all/chromosome/clade/summary_trees_noHyperPrior/relaxed_clock/2.ANT.json' succeeded, but there were warnings you may want to resolve.
Validation successful for local JSON.

Validating schema of '/mnt/c/Users/ktmea/Projects/plague-phylogeography-projects/main/config/auspice_config.json'...
Validation success.
['node_type', 'branch_length', 'timetree_num_date', 'timetree_num_date_confidence', 'ra

Validating that the JSON is internally consistent...
Validation of '/mnt/c/Users/ktmea/Projects/plague-phylogeography-projects/main/beast/all/chromosome/clade/summary_trees_noHyperPrior/relaxed_clock/3.ANT.json' succeeded, but there were warnings you may want to resolve.
Validation successful for local JSON.

Validating schema of '/mnt/c/Users/ktmea/Projects/plague-phylogeography-projects/main/config/auspice_config.json'...
Validation success.
['node_type', 'branch_length', 'timetree_num_date', 'timetree_num_date_confidence', 'rate_beast', 'rate_subs_beast', 'country', 'province', 'country_lat', 'country_lon', 'province_lat', 'province_lon', 'strain', 'branch_major', 'biovar', 'host_human', 'date', 'date_bp']


Validating produced JSON
Validating schema of '/mnt/c/Users/ktmea/Projects/plague-phylogeography-projects/main/beast/all/chromosome/clade/summary_trees_noHyperPrior/relaxed_clock/0.ANT.json'...
Validating that the JSON is internally consistent...
Validation of '/mnt/c/Users/ktmea/Projects/plague-phylogeography-projects/main/beast/all/chromosome/clade/summary_trees_noHyperPrior/relaxed_clock/0.ANT.json' succeeded, but there were warnings you may want to resolve.
Validation successful for local JSON.

Validating schema of '/mnt/c/Users/ktmea/Projects/plague-phylogeography-projects/main/config/auspice_config.json'...
Validation success.


['node_type', 'branch_length', 'timetree_num_date', 'timetree_num_date_confidence', 'rate_beast', 'rate_subs_beast', 'country', 'province', 'country_lat', 'country_lon', 'province_lat', 'province_lon', 'strain', 'branch_major', 'biovar', 'host_human', 'date', 'date_bp']
Validating produced JSON
Validating schema of '/mnt/c/Users/ktmea/Projects/plague-phylogeography-projects/main/beast/all/chromosome/clade/summary_trees_noHyperPrior/relaxed_clock/0.ANT4.json'...
Validating that the JSON is internally consistent...
Validation of '/mnt/c/Users/ktmea/Projects/plague-phylogeography-projects/main/beast/all/chromosome/clade/summary_trees_noHyperPrior/relaxed_clock/0.ANT4.json' succeeded, but there were warnings you may want to resolve.
Validation successful for local JSON.

Validating schema of '/mnt/c/Users/ktmea/Projects/plague-phylogeography-projects/main/config/auspice_config.json'...
Validation success.
['node_type', 'branch_length', 'timetree_num_date', 'timetree_num_date_confidence', '

Validating produced JSON
Validating schema of '/mnt/c/Users/ktmea/Projects/plague-phylogeography-projects/main/beast/all/chromosome/clade/summary_trees_noHyperPrior/relaxed_clock/0.PE.json'...
Validating that the JSON is internally consistent...
Validation of '/mnt/c/Users/ktmea/Projects/plague-phylogeography-projects/main/beast/all/chromosome/clade/summary_trees_noHyperPrior/relaxed_clock/0.PE.json' succeeded, but there were warnings you may want to resolve.
Validation successful for local JSON.

Validating schema of '/mnt/c/Users/ktmea/Projects/plague-phylogeography-projects/main/config/auspice_config.json'...
Validation success.
['node_type', 'branch_length', 'timetree_num_date', 'timetree_num_date_confidence', 'rate_beast', 'rate_subs_beast', 'country', 'province', 'country_lat', 'country_lon', 'province_lat', 'province_lon', 'strain', 'branch_major', 'biovar', 'host_human', 'date', 'date_bp']
Validating produced JSON
Validating schema of '/mnt/c/Users/ktmea/Projects/plague-phyloge